In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir("C:/Users/Administrator/Downloads")

census = pd.read_csv("census_mini.csv", sep=";")
#resultant DF is capitals for each occ by year. 3 dimensional. 
#I can do corelations for each occupation through time, for occupations in one slice of time

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,5,6,7,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
census[census["YEAR"]==1900]

,YEAR,SAMPLE,SERIAL,CBSERIAL,PERNUM,PERWT,EDUC,EDUCD,OCC,OCC1950,OCC1990,INCTOT,FTOTINC,PRESGL,PRENT
0,1900,190001,1,,1,"19,71",,,190,523,,,,"45,3",
1,1900,190001,1,,2,"19,71",,,999,999,,,,0,
2,1900,190001,1,,3,"19,71",,,999,999,,,,0,
3,1900,190001,1,,4,"19,71",,,170,684,,,,"24,9",
4,1900,190001,1,,5,"19,71",,,170,675,,,,"24,9",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3852847,1900,190001,871997,,2,"20,84",,,999,999,,,,0,
3852848,1900,190001,871997,,3,"20,84",,,999,999,,,,0,
3852849,1900,190001,871997,,4,"20,84",,,999,999,,,,0,
3852850,1900,190001,871997,,5,"20,84",,,999,999,,,,0,


In [8]:
occ_cols = {"OCC1950":occ1950_map, "OCC1990":occ1990_map}
occ_col = "OCC1990"
occ_map = {}
capitals = {"EDUC":"Education", "PRESGL":"Status", "FTOTINC":"Affluence"}
#+ INCTOT and FTOTINC and CLASSWKR/OCCSCORE for affluence, EDUC>HIGRADE>EDSCORE90 for education, PRENT also for status
#EDUC, PRESGL (though tied to 60's) useable as is
#exclude on INCs: 9999999 = N/A, EDUC: 00 (N/A or no schooling)
#OCC: https://usa.ipums.org/usa/volii/occ1990.shtml & https://usa.ipums.org/usa-action/variables/OCC1950#codes_section
#years = list(full_census["YEAR"].unique())
year = 1990
#census = full_census#[full_census["YEAR"] == year]
#del full_census
#for occ_col, occ_map in occ_cols.items():
 #   census[occ_col] = census[occ_col].replace([int(i) for i in list(occ_map.keys())], list(occ_map.values()))

census = census[census["EDUC"] != "0"]
census = census[census["EDUC"] != 0]
census = census[census["INCTOT"] != "9999999"]
census = census[census["FTOTINC"] != "9999999"]

In [9]:
for col in ["EDUC", "INCTOT", "FTOTINC","PERWT", "PRENT"]:
    print(col)
    census[col] = census[col].replace(to_replace=r' ', value='', regex=True)
    census = census[census[col] != ""]
    census[col] = census[col].replace(to_replace=r',', value='.', regex=True).astype(float)

EDUC
INCTOT
FTOTINC
PERWT
PRENT


In [13]:
years = list(census["YEAR"].unique())

In [14]:
#STILL NEED TO DO:  na removing, integrate garg occ mapping
def agg_fun(x):
    d = {}
    d['EDUC_mean'] = np.average(x['EDUC'], weights=x["PERWT"])
    d['PRESGL_mean'] = np.average(x['PRESGL'], weights=x["PERWT"])
    #d['PRENT_mean'] = np.average(x['PRENT'], weights=x["PERWT"])
    d['INCTOT_mean'] = np.average(x['INCTOT'], weights=x["PERWT"])
    d['FTOTINC_mean'] = np.average(x['FTOTINC'], weights=x["PERWT"])
    return pd.Series(d, index=['EDUC_mean', 'PRESGL_mean', 'PRENT_mean', 'FTOTINC_mean', 'INCTOT_mean'])


In [15]:
for occ_col, occ_map in occ_cols.items():
    print(occ_col)
    #census[occ_col] = census[occ_col].replace([int(i) for i in list(occ_map.keys())], list(occ_map.values()))
    for year in years[::-1]:
        print(year)
        cen = census[census["YEAR"] == year]
        try:
            #[["PRESGL"] = cen["PRESGL"].replace(to_replace=r',', value='.', regex=True).astype('float64')
            cen[["PRESGL","PRENT"]] = cen[["PRESGL","PRENT"]].replace(to_replace=r',', value='.', regex=True).astype('float64')
        except:
            pass
        agg_census = cen.groupby(occ_col).apply(agg_fun)
       # agg_census[occ_col] = agg_census[occ_col].replace([int(i) for i in list(occ_map.keys())], list(occ_map.values()))
        occ_new = {}
        for key, val in occ_map.items():
            occ_new[int(key)] = val
        agg_census.rename(index=occ_new, inplace=True)
        agg_census.to_csv("census_" + str(year) + "_" + occ_col + ".csv")
        del cen
        del agg_census

    

OCC1950
2020


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


2010
2000
1990
1980
1970
1960
1950
OCC1990
2020
2010
2000
1990
1980
1970
1960
1950


In [33]:
occ_new = {}
for key, val in occ_map.items():
    occ_new[int(key)] = val
agg_census.rename(index=occ_new, inplace=True)
agg_census

,EDUC_mean,PRESGL_mean,PRENT_mean,FTOTINC_mean,INCTOT_mean
OCC1950,,,,,
Accountants and auditors,9.837828,56.7,65.400000,158440.621475,8.395428e+04
Actors and actresses,8.358561,55.0,57.600000,235943.348622,3.870080e+04
Airplane pilots and navigators,9.248422,70.1,61.000000,225149.302006,1.605334e+05
Architects,10.168636,70.5,73.200000,194244.732856,9.578483e+04
Artists and art teachers,8.600548,56.2,52.400000,210819.770564,4.687984e+04
...,...,...,...,...,...
Fishermen and oystermen,6.318299,30.2,34.100000,579108.824402,4.818489e+04
"Gardeners, except farm and groundskeepers",5.944542,22.5,28.600000,676074.121400,2.479714e+04
"Lumbermen, raftsmen, and woodchoppers",5.796627,26.4,33.300000,348810.421784,3.651930e+04


In [ ]:
mapping = pd.read_csv("G:/My Drive/KU/Thesis/data/garg/occupatons_map.csv", index_col=None)
occu_conv = {row["1950 basis"]:row["Single words"] for row in mapping.iterrows()}
capitals = {"PRESGL": "status", "INCTOT":"affluence", "EDUC":"education", "EDUC":"cultivation"}
averages = {}
for occ in occu_conv.keys():
    averages[occ] = {}
    for capital in capitals.keys():
        averages[occ][capital + "_mean"] = census[census.loc[:,"occ1950"]==occ].loc[:,capital].mean()
        averages[occ][capital + "_median"] = census[census.loc[:,"occ1950"]==occ].loc[:,capital].median()
        #gcbs[gcbs.loc[:,el].isin([2,3])].loc[:,capital].mean()

# GBCS

In [ ]:
gcbs = pd.read_stata("G:/My Drive/KU/Thesis/data/gbcs_stata/stata11/gbcs_cleaned_eul.dta")#, convert_categoricals=False)

In [ ]:
activities_conv = {"carts":"gallery", "cbingo":"bingo", "csportw":"sport", "ctheatre":"theatre", 
                "copera":"opera", "cdance":"dance","cmags":"magazine", "cdiy":"diy", 
                "cpub":"pub", "cclassic":"classical", "cshop":"shopping", "cresta":"restaurant",
                "ctv":"tv", "cbooks":"reading", "cgig":"gig"}
music_conv = {"mmetal":"metal","mreggae":"reggae", "mjazz":"jazz", "mpop":"pop", "mfolk":"folk",
             "mclassic":"classical", "mrock":"rock", "mcountry":"country"}
conv = dict(activities_conv, **music_conv)
capitals = {"hhincome": "affluence", "educ":"education", "chigh":"cultivation"} 
act_likert_conv = {"Never":0, "Rarely":1, "Sometimes":2, "Often":3}
music_likert_conv = {"Dislike a lot":1, "Dislike":2, "Neither":3, "Like":4, "Like a lot":5}
for el in activities_conv.keys():
    gcbs.loc[:,el]=  gcbs.loc[:,el].replace(act_likert_conv)
for el in music_conv.keys():
    gcbs.loc[:,el]=  gcbs.loc[:,el].replace(music_likert_conv)

In [ ]:
#gcbs["hhincome"] = gcbs["hhincome"].apply(lambda x: 2.5+5*(x-1)) 
sorted(list(gcbs["hhincome"].unique())[:-1])
hhincome_conv = {'Over £200,000':300000,
 'Under £5,000':2500,
 '£10,000 - £14,999':12500,
 '£100,000 - £149,999':125000,
 '£15,000 - £19,999':17500,
 '£150,000 - £199,999':175000,
 '£20,000 - £24,999':22500,
 '£25,000 - £29,999':27500,
 '£30,000 - £34,999':32500,
 '£35,000 - £39,999':37500,
 '£40,000 - £44,999':42500,
 '£45,000 - £49,999':47500,
 '£5,000 - £10,000':7500,
 '£50,000 - £54,999':52500,
 '£55,000 - £59,999':57500,
 '£60,000 - £64,999':62500,
 '£65,000 - £69,999':67500,
 '£70,000 - £74,999':72500,
 '£75,000 - £79,999':77500,
 '£80,000 - £84,999':82500,
 '£85,000 - £89,999':87500,
 '£90,000 - £94,999':92500,
 '£95,000 - £99,999':97500, 
 np.nan: 0}
educ_conv = {
    '1.  Did not complete GCSE/CSE/O-levels':1,
 '2. A-levels/Scottish Highers':2,
 '3. Completed GCSE/CSE/O-levels':3,
 '4. Completed post-16 vocational course':4,
 '5. I am still in education':5,
 '6. Postgraduate degree':6,
 '7. Undergraduate degree':7
}

gcbs.loc[:,"educ"]=  gcbs.loc[:,"educ"].replace(educ_conv)
gcbs.loc[:,"hhincome"]=  gcbs.loc[:,"hhincome"].replace(hhincome_conv)

In [ ]:
gcbs.loc[:,"hhincome"].unique()

In [ ]:
averages = {el : {} for el in conv.keys()}
for el in activities_conv.keys():
    averages[el] = {}
    for capital in capitals.keys():
        averages[el][capital + "_mean"] = gcbs[gcbs.loc[:,el].isin([2,3])].loc[:,capital].mean()
        averages[el][capital + "_median"] = gcbs[gcbs.loc[:,el].isin([2,3])].loc[:,capital].median()
                                                                         
for el in music_conv.keys():
    averages[el] = {}
    for capital in capitals.keys():
        averages[el][capital + "_mean"] = gcbs[gcbs.loc[:,el].isin([4,5])].loc[:,capital].mean()
        averages[el][capital + "_median"] = gcbs[gcbs.loc[:,el].isin([4,5])].loc[:,capital].median()   

In [ ]:
df
df = pd.DataFrame(averages)
df = df.rename_axis("activity")
#df.set_index("activity")
dft = df.T
#gcbs.shape


In [ ]:
dft

In [ ]:
dft.to_csv(f"G:/My Drive/KU/Thesis/data/survey_data/gbcs_averages.csv")
pd.read_csv("G:/My Drive/KU/Thesis/data/survey_data/gbcs_averages.csv", index_col= 0)#,index_col="activity")

In [ ]:
gss = pd.read_stata("G:/My Drive/KU/Thesis/data/GSS_stata/gss7221_r1b.dta", convert_categoricals=False)

In [ ]:
activities = ["attsprts", "visitart", "makeart", "autorace", "camping", "garden", "dance", "gomusic", "huntfish", "perform", "dosports", "seemovie", "usevcr", "plymusic", "readfict", "popmusic", "drama", "relart", "volarts"] 
music_genres = ["bigband","blugrass", "country", "blues", "musicals", "classicl", "folk", "gospel", "jazz", "latin", "moodeasy", "newage", "opera", "rap", "reggae", "conrock", "oldies", "hvymetal"]
cont = ["gamegn"]
domains = activities + music_genres
capitals = ["educ", "prestg80", "rincome", "wealth", "class"]
gss[activities] = gss[activities].apply(lambda x: abs(x-2))
 #class: 1 lower, 2 working, 3 middle, 4 upper, 5 no
gss[["educ", "prestige", "rincome", "wealth"]].head() #occ, occ80, occ10 industry, rincome77, rincome82, rincome86, rincome91, rincome98

#1= yes, 2= no, change to 1 = yes, 0 = no
means = {activity: {cap + "_mean": 0 for cap in capitals} for activity in domains}
medians = {activity: {cap + "_median": 0 for cap in capitals} for activity in domains}
averages = means  | medians
#gss[activities].describe()
for activity in domains:
    df = gss[gss[activity]== 1]
    print(activity, gss[-gss[activity].isna()]["year"].unique())
    for cap in capitals:
        averages[activity][cap + "_median"] = df[cap].median()
        averages[activity][cap + "_mean"] = df[cap].mean()
#for cap in capitals:
 #   print(cap, gss[-gss[cap].isna()]["year"].unique())
#gss[capitals].describe()
df = pd.DataFrame(averages).T
df.index.name = "capital"
df.to_csv(f"G:/My Drive/KU/Thesis/data/survey_data/gss_averages.csv")
pd.read_csv(f"G:/My Drive/KU/Thesis/data/survey_data/gss_averages.csv", index_col = "capital")
#music_genres_conv = {"bigband": "big band","blugrass": "bluegrass", "country": "country", "blues": "blues", "musicals": "musicals", "classicl": "classical", "folk": "folk", "gospel": "gospel", "jazz": "jazz", "latin": ["salsa", "samba", "bachata", "kilomba", "tango", "rumba", "bolero"], "moodeasy": "easy-listening", "newage": ["new-age", "newage"], "opera": "opera", "rap": "rap", "reggae": "reggae", "conrock": "rock", "oldies": "oldies", "hvymetal": "heavy-metal"}
#activities_conv = {"attsprts":"sports", "visitart":"exhibition", "makeart":"art", "autorace":"autosport", "camping":"camping", "garden":"gardening", "dance":["dancing","dance"], "gomusic":["concert", "gig"] "huntfish":["hunting","fishing"], "perform":[], "dosports":["sports", "sport"], "seemovie":[], "usevcr":[], "plymusic":[], "readfict":"reading", "popmusic":"pop", "drama":[], "relart":[], "volarts":[]}
#capitals = ["educ":"education", "prestg80", "rincome", "wealth", "class"]

### Cleaning occupational map data

In [ ]:
s= "000	Accountants and auditors	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	000	Accountants and auditors	X	·	X	X	X	X	X	X	X	X 001	Actors and actresses	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	·	·	X	X	X	001	Actors and actresses	X	X	X	X	X	X	X	X	X	X 002	Airplane pilots and navigators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	002	Airplane pilots and navigators	X	X	X	X	X	X	X	·	·	· 003	Architects	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	003	Architects	X	X	X	X	X	X	X	X	X	X 004	Artists and art teachers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	004	Artists and art teachers	X	X	X	X	X	X	X	X	X	X 005	Athletes	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	005	Athletes	X	X	X	X	X	X	X	X	X	X 006	Authors	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	006	Authors	X	X	X	X	X	X	X	X	X	X 007	Chemists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	007	Chemists	X	X	X	X	X	X	X	X	X	X 008	Chiropractors	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	008	Chiropractors	X	X	X	X	X	·	·	·	·	· 009	Clergymen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	009	Clergymen	X	X	X	X	X	X	X	X	X	X 010	College presidents and deans	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	010	College presidents and deans	X	·	X	X	X	X	X	X	X	X Professors and instructors:																											Professors and instructors: 012	Agricultural sciences	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	012	Agricultural sciences	X	·	X	X	·	·	X	·	·	· 013	Biological sciences	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	013	Biological sciences	X	·	X	X	X	X	X	·	·	· 014	Chemistry	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	014	Chemistry	X	·	X	X	X	X	X	·	·	· 015	Economics	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	015	Economics	X	·	X	·	·	·	·	·	·	· 016	Engineering	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	016	Engineering	X	·	X	X	·	X	X	·	·	· 017	Geology and geophysics	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	017	Geology and geophysics	X	·	X	·	·	X	X	X	·	· 018	Mathematics	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	018	Mathematics	X	·	X	X	·	X	X	X	X	X 019	Medical sciences	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	019	Medical sciences	X	·	X	X	X	X	X	·	X	· 023	Physics	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	023	Physics	X	·	X	X	X	X	X	·	·	· 024	Psychology	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	024	Psychology	X	·	X	·	·	·	·	·	·	· 025	Statistics	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	025	Statistics	X	·	X	·	·	·	·	·	·	· 026	Natural science (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	026	Natural science (n.e.c.)	X	·	·	·	·	X	X	·	X	· Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 027	Social sciences (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	027	Social sciences (n.e.c.)	X	·	X	X	X	X	X	X	X	· 028	Non-scientific subjects	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	028	Non-scientific subjects	X	·	X	X	X	X	X	X	X	X 029	Subject not specified	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	029	Subject not specified	X	X	X	X	X	X	X	X	X	X 031	Dancers and dancing teachers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	031	Dancers and dancing teachers	X	X	X	X	X	X	X	X	X	X 032	Dentists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	032	Dentists	X	X	X	X	X	X	X	X	X	X 033	Designers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	033	Designers	X	X	X	X	X	X	X	X	X	X 034	Dietitians and nutritionists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	034	Dietitians and nutritionists	X	·	X	X	X	X	·	·	·	· 035	Draftsmen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	035	Draftsmen	X	X	X	X	X	X	X	X	X	X 036	Editors and reporters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	036	Editors and reporters	X	X	X	X	X	X	X	X	X	X 041	Engineers, aeronautical	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	041	Engineers, aeronautical	X	·	X	X	·	X	·	·	·	· 042	Engineers, chemical	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	042	Engineers, chemical	X	X	X	X	X	X	·	·	·	· 043	Engineers, civil	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	043	Engineers, civil	X	X	X	X	X	X	X	X	X	X 044	Engineers, electrical	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	044	Engineers, electrical	X	X	X	X	X	X	·	·	·	· 045	Engineers, industrial	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	045	Engineers, industrial	X	X	X	X	X	·	·	·	·	· 046	Engineers, mechanical	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	046	Engineers, mechanical	X	X	X	X	X	X	X	X	X	X 047	Engineers, metallurgical, metallurgists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	047	Engineers, metallurgical, metallurgists	X	X	X	X	X	X	X	·	·	· 048	Engineers, mining	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	048	Engineers, mining	X	·	X	X	X	X	X	X	X	· 049	Engineers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	049	Engineers (n.e.c.)	X	·	X	X	X	X	X	X	X	· 051	Entertainers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	051	Entertainers (n.e.c.)	X	X	X	X	X	X	X	X	X	X 052	Farm and home management advisors	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	052	Farm and home management advisors	X	X	X	X	X	·	·	·	X	· 053	Foresters and conservationists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	053	Foresters and conservationists	X	·	X	X	X	X	X	X	X	X 054	Funeral directors and embalmers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	054	Funeral directors and embalmers	X	X	X	X	X	X	X	X	X	X 055	Lawyers and judges	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	055	Lawyers and judges	X	X	X	X	X	X	X	X	X	X 056	Librarians	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	056	Librarians	X	X	X	X	X	X	X	X	X	· 057	Musicians and music teachers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	057	Musicians and music teachers	X	X	X	X	X	X	X	X	X	X Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 058	Nurses, professional	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	058	Nurses, professional	X	X	X	X	X	X	X	X	·	· 059	Nurses, student professional	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	059	Nurses, student professional	X	·	X	X	X	X	·	·	·	· 061	Agricultural scientists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	061	Agricultural scientists	X	·	X	X	X	X	X	·	·	· 062	Biological scientists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	062	Biological scientists	X	·	X	X	X	X	X	X	X	X 063	Geologists and geophysicists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	063	Geologists and geophysicists	X	·	X	X	X	X	X	X	·	· 067	Mathematicians	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	067	Mathematicians	X	·	X	·	X	·	X	·	·	· 068	Physicists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	068	Physicists	X	·	X	X	X	X	·	·	·	· 069	Miscellaneous natural scientists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	069	Miscellaneous natural scientists	X	·	X	X	X	X	X	·	X	· 070	Optometrists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	070	Optometrists	X	X	X	X	X	X	·	·	·	· 071	Osteopaths	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	071	Osteopaths	X	X	X	X	X	X	·	·	·	· 072	Personnel and labor relations workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	072	Personnel and labor relations workers	X	·	X	X	X	·	·	·	·	· 073	Pharmacists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	073	Pharmacists	X	X	X	X	X	X	X	X	X	X 074	Photographers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	074	Photographers	X	X	X	X	X	X	X	X	X	X 075	Physicians and surgeons	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	075	Physicians and surgeons	X	X	X	X	X	X	X	X	X	X 076	Radio operators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	076	Radio operators	X	X	X	X	X	X	·	·	·	· 077	Recreation and group workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	077	Recreation and group workers	X	·	X	X	X	X	X	·	·	· 078	Religious workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	078	Religious workers	X	X	X	X	X	X	X	X	X	X 079	Social and welfare workers, except group	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	079	Social and welfare workers, except group	X	X	X	X	X	X	X	·	·	· 081	Economists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	081	Economists	X	·	X	X	·	·	·	·	·	· 082	Psychologists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	082	Psychologists	X	·	X	X	X	·	X	·	·	· 083	Statisticians and actuaries	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	083	Statisticians and actuaries	X	·	X	X	X	X	X	·	·	· 084	Miscellaneous social scientists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	084	Miscellaneous social scientists	X	·	X	X	X	X	X	·	·	· 091	Sports instructors and officials	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	091	Sports instructors and officials	X	X	X	X	X	X	X	X	·	· 092	Surveyors	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	092	Surveyors	X	X	X	X	X	X	X	X	X	X 093	Teachers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	093	Teachers (n.e.c.)	X	X	X	X	X	X	X	X	X	X Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 094	Technicians, medical and dental	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	094	Technicians, medical and dental	X	·	X	X	X	X	X	·	·	· 095	Technicians, testing	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	095	Technicians, testing	X	X	X	X	X	X	X	·	·	· 096	Technicians (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	096	Technicians (n.e.c.)	X	X	X	X	X	X	X	X	·	· 097	Therapists and healers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	097	Therapists and healers (n.e.c.)	X	X	X	X	X	X	X	X	X	X 098	Veterinarians	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	098	Veterinarians	X	X	X	X	X	X	X	X	X	· 099	Professional, technical and kindred workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	099	Professional, technical and kindred workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X Farmers:																											Farmers: 100	Farmers (owners and tenants)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	100	Farmers (owners and tenants)	X	X	X	X	X	X	X	X	X	X 123	Farm managers	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	123	Farm managers	X	X	X	X	X	X	X	X	X	X Managers, Officials, and Proprietors:																											Managers, Officials, and Proprietors: 200	Buyers and department heads, store	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	200	Buyers and department heads, store	X	X	X	X	X	X	X	·	X	· 201	Buyers and shippers, farm products	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	201	Buyers and shippers, farm products	X	X	X	X	X	X	X	X	X	X 203	Conductors, railroad	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	203	Conductors, railroad	X	X	X	X	X	X	X	X	X	X 204	Credit men	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	204	Credit men	X	X	X	X	X	X	·	·	·	· 205	Floormen and floor managers, store	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	205	Floormen and floor managers, store	X	X	X	X	X	X	X	·	·	· 210	Inspectors, public administration	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	210	Inspectors, public administration	X	X	X	X	X	X	X	X	X	X 230	Managers and superintendents, building	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	230	Managers and superintendents, building	X	X	X	X	X	X	X	X	X	X 240	Officers, pilots, pursers and engineers, ship	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	240	Officers, pilots, pursers and engineers, ship	X	X	X	X	X	X	X	X	X	X 250	Officials and administrators (n.e.c.), public administration	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	250	Officials and administrators (n.e.c.), public administration	X	X	X	X	X	X	X	X	X	X 260	Officials, lodge, society, union, etc.	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	260	Officials, lodge, society, union, etc.	X	X	X	X	X	X	X	·	·	· 270	Postmasters	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	270	Postmasters	X	X	X	X	X	X	X	X	X	X 280	Purchasing agents and buyers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	280	Purchasing agents and buyers (n.e.c.)	X	X	X	X	X	X	X	·	·	· 290	Managers, officials, and proprietors (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	290	Managers, officials, and proprietors (n.e.c.)	X	X	X	X	X	X	X	X	X	X Clerical and Kindred																											Clerical and Kindred 300	Agents (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	300	Agents (n.e.c.)	X	X	X	X	X	X	X	X	X	X Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 301	Attendants and assistants, library	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	301	Attendants and assistants, library	X	X	X	X	X	X	X	X	·	· 302	Attendants, physician's and dentist's office	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	302	Attendants, physician's and dentist's office	X	X	X	X	X	X	X	·	·	· 304	Baggagemen, transportation	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	304	Baggagemen, transportation	X	X	X	X	X	X	X	X	X	X 305	Bank tellers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	305	Bank tellers	X	·	X	X	X	X	X	X	X	X 310	Bookkeepers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	310	Bookkeepers	X	X	X	X	X	X	X	X	X	X 320	Cashiers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	320	Cashiers	X	·	X	X	X	X	X	X	X	X 321	Collectors, bill and account	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	321	Collectors, bill and account	X	X	X	X	X	X	X	X	X	X 322	Dispatchers and starters, vehicle	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	322	Dispatchers and starters, vehicle	X	·	X	X	X	X	X	X	·	· 325	Express messengers and railway mail clerks	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	325	Express messengers and railway mail clerks	X	X	X	X	X	X	X	X	X	· 335	Mail carriers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	335	Mail carriers	X	X	X	X	X	X	X	X	X	X 340	Messengers and office boys	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	340	Messengers and office boys	X	X	X	X	X	X	X	X	X	X 341	Office machine operators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	341	Office machine operators	X	X	X	X	X	X	X	·	·	· 342	Shipping and receiving clerks	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	342	Shipping and receiving clerks	X	X	X	X	X	X	X	X	X	· 350	Stenographers, typists, and secretaries	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	350	Stenographers, typists, and secretaries	X	X	X	X	X	X	X	X	X	· 360	Telegraph messengers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	360	Telegraph messengers	X	X	X	X	X	X	X	X	·	X 365	Telegraph operators	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	365	Telegraph operators	X	X	X	X	X	X	X	X	X	X 370	Telephone operators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	370	Telephone operators	X	X	X	X	X	X	X	·	·	· 380	Ticket, station, and express agents	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	380	Ticket, station, and express agents	X	X	X	X	X	X	X	X	X	X 390	Clerical and kindred workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	390	Clerical and kindred workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X Sales workers:																											Sales workers: 400	Advertising agents and salesmen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	400	Advertising agents and salesmen	X	X	X	X	X	X	X	X	·	· 410	Auctioneers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	410	Auctioneers	X	X	X	X	X	X	X	X	X	X 420	Demonstrators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	420	Demonstrators	X	X	X	X	X	X	·	·	·	· 430	Hucksters and peddlers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	430	Hucksters and peddlers	X	X	X	X	X	X	X	X	X	X 450	Insurance agents and brokers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	450	Insurance agents and brokers	X	X	X	X	X	X	X	X	X	X Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 460	Newsboys	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	460	Newsboys	X	X	X	X	X	X	X	X	X	X 470	Real estate agents and brokers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	470	Real estate agents and brokers	X	X	X	X	X	X	X	X	X	X 480	Stock and bond salesmen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	480	Stock and bond salesmen	X	X	X	X	X	X	X	·	·	· 490	Salesmen and sales clerks (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	490	Salesmen and sales clerks (n.e.c.)	X	X	X	X	X	X	X	X	X	X Craftsmen:																											Craftsmen: 500	Bakers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	500	Bakers	X	X	X	X	X	X	X	X	X	X 501	Blacksmiths	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	501	Blacksmiths	X	X	X	X	X	X	X	X	X	X 502	Bookbinders	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	502	Bookbinders	X	·	X	X	X	X	X	X	X	X 503	Boilermakers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	503	Boilermakers	X	X	X	X	X	X	X	X	X	X 504	Brickmasons, stonemasons, and tile setters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	504	Brickmasons, stonemasons, and tile setters	X	X	X	X	X	X	X	X	X	X 505	Cabinetmakers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	505	Cabinetmakers	X	X	X	X	X	X	X	X	X	X 510	Carpenters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	510	Carpenters	X	X	X	X	X	X	X	X	X	X 511	Cement and concrete finishers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	511	Cement and concrete finishers	X	X	X	X	X	X	X	·	X	· 512	Compositors and typesetters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	512	Compositors and typesetters	X	X	X	X	X	X	X	X	X	X 513	Cranemen, derrickmen, and hoistmen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	513	Cranemen, derrickmen, and hoistmen	X	X	X	X	X	X	X	·	·	· 514	Decorators and window dressers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	514	Decorators and window dressers	X	X	X	X	X	X	X	X	·	X 515	Electricians	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	515	Electricians	X	X	X	X	X	X	X	X	·	· 520	Electrotypers and stereotypers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	520	Electrotypers and stereotypers	X	X	X	X	X	X	X	X	X	· 521	Engravers, except photoengravers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	521	Engravers, except photoengravers	X	X	X	X	X	X	X	X	X	X 522	Excavating, grading, and road machinery operators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	522	Excavating, grading, and road machinery operators	X	·	X	X	X	X	X	·	·	· 523	Foremen (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	523	Foremen (n.e.c.)	X	X	X	X	X	X	X	X	X	X 524	Forgemen and hammermen	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	524	Forgemen and hammermen	X	·	X	X	X	X	X	X	X	X 525	Furriers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	525	Furriers	X	X	X	X	X	X	X	X	X	X 530	Glaziers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	530	Glaziers	X	X	X	X	X	X	X	X	X	· 531	Heat treaters, annealers, temperers	·	·	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	531	Heat treaters, annealers, temperers	X	X	X	X	X	X	X	X	·	· Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 532	Inspectors, scalers, and graders, log and lumber	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	532	Inspectors, scalers, and graders, log and lumber	X	X	X	X	X	X	X	X	X	X 533	Inspectors (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	533	Inspectors (n.e.c.)	X	X	X	X	X	X	X	X	X	X 534	Jewelers, watchmakers, goldsmiths, and silversmiths	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	534	Jewelers, watchmakers, goldsmiths, and silversmiths	X	X	X	X	X	X	X	X	X	X 535	Job setters, metal	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	535	Job setters, metal	X	·	X	X	X	·	X	·	·	· 540	Linemen and servicemen, telegraph, telephone, and power	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	540	Linemen and servicemen, telegraph, telephone, and power	X	X	X	X	X	X	X	X	·	· 541	Locomotive engineers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	541	Locomotive engineers	X	X	X	X	X	X	X	X	X	X 542	Locomotive firemen	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	542	Locomotive firemen	X	X	X	X	X	X	X	X	X	· 543	Loom fixers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	543	Loom fixers	X	X	X	X	X	X	X	X	X	· 544	Machinists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	544	Machinists	X	X	X	X	X	X	X	X	X	X 545	Mechanics and repairmen, airplane	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	545	Mechanics and repairmen, airplane	X	X	X	X	·	·	·	·	·	· 550	Mechanics and repairmen, automobile	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	550	Mechanics and repairmen, automobile	X	X	X	X	X	X	·	·	·	· 551	Mechanics and repairmen, office machine	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	551	Mechanics and repairmen, office machine	X	·	X	X	X	X	·	·	·	· 552	Mechanics and repairmen, radio and television	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	552	Mechanics and repairmen, radio and television	X	·	X	X	·	·	·	·	·	· 553	Mechanics and repairmen, railroad and car shop	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	553	Mechanics and repairmen, railroad and car shop	X	X	X	X	X	X	X	X	·	· 554	Mechanics and repairmen (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	554	Mechanics and repairmen (n.e.c.)	X	X	X	X	X	X	X	X	X	X 555	Millers, grain, flour, feed, etc.	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	555	Millers, grain, flour, feed, etc.	X	X	X	X	X	X	X	X	X	X 560	Millwrights	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	560	Millwrights	X	X	X	X	X	X	X	X	X	X 561	Molders, metal	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	561	Molders, metal	X	X	X	X	X	X	X	X	X	X 562	Motion picture projectionists	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	·	·	·	·	X	X	X	X	X	X	562	Motion picture projectionists	X	X	X	X	X	X	·	·	·	· 563	Opticians and lens grinders and polishers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	563	Opticians and lens grinders and polishers	X	X	X	X	X	X	X	X	X	X 564	Painters, construction and maintenance	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	564	Painters, construction and maintenance	X	X	X	X	X	X	X	X	X	X 565	Paperhangers	·	·	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	X	X	·	·	·	·	X	565	Paperhangers	X	X	X	X	X	X	X	X	X	X 570	Pattern and model makers, except paper	·	·	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	570	Pattern and model makers, except paper	X	X	X	X	X	X	X	X	X	X 571	Photoengravers and lithographers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	571	Photoengravers and lithographers	X	X	X	X	X	X	X	X	X	X 572	Piano and organ tuners and repairmen	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	572	Piano and organ tuners and repairmen	X	X	X	X	X	X	X	X	X	· Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 573	Plasterers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	573	Plasterers	X	X	X	X	X	X	X	X	X	X 574	Plumbers and pipe fitters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	574	Plumbers and pipe fitters	X	X	X	X	X	X	X	X	X	X 575	Pressmen and plate printers, printing	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	575	Pressmen and plate printers, printing	X	X	X	X	X	X	X	X	X	X 580	Rollers and roll hands, metal	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	580	Rollers and roll hands, metal	X	X	X	X	X	X	X	X	X	X 581	Roofers and slaters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	581	Roofers and slaters	X	X	X	X	X	X	X	X	X	X 582	Shoemakers and repairers, except factory	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	582	Shoemakers and repairers, except factory	X	X	X	X	X	X	X	X	X	X 583	Stationary engineers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	583	Stationary engineers	X	X	X	X	X	X	X	X	X	X 584	Stone cutters and stone carvers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	584	Stone cutters and stone carvers	X	X	X	X	X	X	X	X	X	X 585	Structural metal workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	585	Structural metal workers	X	X	X	X	X	X	X	X	X	X 590	Tailors and tailoresses	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	590	Tailors and tailoresses	X	X	X	X	X	X	X	X	X	X 591	Tinsmiths, coppersmiths, and sheet metal workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	591	Tinsmiths, coppersmiths, and sheet metal workers	X	X	X	X	X	X	X	X	X	X 592	Tool makers, and die makers and setters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	592	Tool makers, and die makers and setters	X	X	X	X	X	X	X	X	X	X 593	Upholsterers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	593	Upholsterers	X	X	X	X	X	X	X	X	X	X 594	Craftsmen and kindred workers (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	594	Craftsmen and kindred workers (n.e.c.)	X	·	X	X	X	X	X	X	X	X 595	Members of the armed services	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	595	Members of the armed services	X	X	X	X	X	X	X	X	X	X Operatives:																											Operatives: 600	Apprentice auto mechanics	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	600	Apprentice auto mechanics	X	·	X	X	X	·	·	·	·	· 601	Apprentice bricklayers and masons	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	601	Apprentice bricklayers and masons	X	·	X	X	X	X	X	X	X	· 602	Apprentice carpenters	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	602	Apprentice carpenters	X	X	X	X	X	X	X	X	X	X 603	Apprentice electricians	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	603	Apprentice electricians	X	X	X	X	X	X	X	·	·	· 604	Apprentice machinists and toolmakers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	604	Apprentice machinists and toolmakers	X	X	X	X	X	X	X	X	X	· 605	Apprentice mechanics, except auto	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	605	Apprentice mechanics, except auto	X	·	X	X	X	X	X	X	X	· 610	Apprentice plumbers and pipe fitters	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	610	Apprentice plumbers and pipe fitters	X	X	X	X	X	X	X	X	X	· 611	Apprentices, building trades (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	611	Apprentices, building trades (n.e.c.)	X	X	X	X	X	X	X	X	X	X 612	Apprentices, metalworking trades (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	612	Apprentices, metalworking trades (n.e.c.)	X	·	X	X	X	X	X	X	X	· Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 613	Apprentices, printing trades	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	613	Apprentices, printing trades	X	X	X	X	X	X	X	X	X	· 614	Apprentices, other specified trades	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	614	Apprentices, other specified trades	X	X	X	X	X	X	X	X	X	X 615	Apprentices, trade not specified	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	615	Apprentices, trade not specified	X	X	X	X	X	X	X	X	X	X 620	Asbestos and insulation workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	620	Asbestos and insulation workers	X	X	X	X	X	X	·	·	·	· 621	Attendants, auto service and parking	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	621	Attendants, auto service and parking	X	X	X	X	X	X	·	·	·	· 622	Blasters and powdermen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	622	Blasters and powdermen	X	X	X	X	X	X	X	X	·	X 623	Boatmen, canalmen, and lock keepers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	623	Boatmen, canalmen, and lock keepers	X	X	X	X	X	X	X	X	X	X 624	Brakemen, railroad	·	·	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	624	Brakemen, railroad	X	X	X	X	X	X	X	X	X	X 625	Bus drivers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	625	Bus drivers	X	·	X	X	X	X	X	X	X	X 630	Chainmen, rodmen, and axmen, surveying	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	630	Chainmen, rodmen, and axmen, surveying	X	X	X	X	X	X	X	·	·	· 631	Conductors, bus and street railway	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	631	Conductors, bus and street railway	X	X	X	X	X	X	X	X	X	· 632	Deliverymen and routemen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	632	Deliverymen and routemen	X	X	X	X	X	X	X	X	X	X 633	Dressmakers and seamstresses, except factory	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	633	Dressmakers and seamstresses, except factory	X	X	X	X	X	X	X	X	X	· 634	Dyers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	634	Dyers	X	X	X	X	X	X	X	X	X	X 635	Filers, grinders, and polishers, metal	X	X	·	·	·	·	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	635	Filers, grinders, and polishers, metal	X	X	X	X	X	X	X	X	X	X 640	Fruit, nut, and vegetable graders, and packers, except factory	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	640	Fruit, nut, and vegetable graders, and packers, except factory	X	X	X	X	X	X	X	X	·	· 641	Furnacemen, smeltermen and pourers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	641	Furnacemen, smeltermen and pourers	X	X	X	X	X	X	X	X	X	X 642	Heaters, metal	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	642	Heaters, metal	X	X	X	X	X	X	X	X	X	X 643	Laundry and dry cleaning operatives	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	643	Laundry and dry cleaning operatives	X	X	X	X	X	X	X	X	X	· 644	Meat cutters, except slaughter and packing house	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	644	Meat cutters, except slaughter and packing house	X	X	X	X	X	X	X	X	X	X 645	Milliners	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	645	Milliners	X	X	X	X	X	X	X	X	X	X 650	Mine operatives and laborers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	650	Mine operatives and laborers	X	X	X	X	X	X	X	X	X	X 660	Motormen, mine, factory, logging camp, etc.	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	660	Motormen, mine, factory, logging camp, etc.	X	X	X	X	X	X	·	·	·	· 661	Motormen, street, subway, and elevated railway	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	661	Motormen, street, subway, and elevated railway	X	X	X	X	X	X	·	X	X	· 662	Oilers and greaser, except auto	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	662	Oilers and greaser, except auto	X	X	X	X	X	X	X	·	·	· Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 670	Painters, except construction or maintenance	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	670	Painters, except construction or maintenance	X	X	X	X	X	X	X	X	X	X 671	Photographic process workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	671	Photographic process workers	X	X	X	X	X	X	X	X	·	X 672	Power station operators	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	672	Power station operators	X	X	X	X	X	X	X	·	·	· 673	Sailors and deck hands	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	673	Sailors and deck hands	X	X	X	X	X	X	X	X	X	X 674	Sawyers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	674	Sawyers	X	X	X	X	X	X	X	X	X	X 675	Spinners, textile	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	675	Spinners, textile	X	·	X	X	X	X	X	X	X	X 680	Stationary firemen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	680	Stationary firemen	X	X	X	X	X	X	X	X	X	X 681	Switchmen, railroad	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	681	Switchmen, railroad	X	X	X	X	X	X	X	X	X	X 682	Taxicab drivers and chauffers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	682	Taxicab drivers and chauffers	X	·	X	X	X	X	X	X	X	X 683	Truck and tractor drivers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	683	Truck and tractor drivers	X	X	X	X	X	X	X	X	X	X 684	Weavers, textile	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	684	Weavers, textile	X	·	X	X	X	X	X	X	X	X 685	Welders and flame cutters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	685	Welders and flame cutters	X	X	X	X	X	X	X	X	X	X 690	Operative and kindred workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	690	Operative and kindred workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X Service Workers (private household):																											Service Workers (private household): 700	Housekeepers, private household	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	700	Housekeepers, private household	X	X	X	X	X	X	X	X	X	· 710	Laundresses, private household	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	710	Laundresses, private household	X	X	X	X	X	X	X	X	X	X 720	Private household workers (n.e.c.)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	720	Private household workers (n.e.c.)	X	X	X	X	X	X	X	X	X	X Service Workers (not household):																											Service Workers (not household): 730	Attendants, hospital and other institution	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	730	Attendants, hospital and other institution	X	X	X	X	X	X	X	X	X	X 731	Attendants, professional and personal service (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	731	Attendants, professional and personal service (n.e.c.)	X	X	X	X	X	X	X	X	X	· 732	Attendants, recreation and amusement	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	732	Attendants, recreation and amusement	X	X	X	X	X	X	X	X	·	X 740	Barbers, beauticians, and manicurists	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	740	Barbers, beauticians, and manicurists	X	X	X	X	X	X	X	X	X	X 750	Bartenders	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	750	Bartenders	X	X	X	X	X	X	X	X	X	X 751	Bootblacks	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	751	Bootblacks	X	X	X	X	X	X	X	X	X	· 752	Boarding and lodging house keepers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	752	Boarding and lodging house keepers	X	X	X	X	X	X	X	X	X	X Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 753	Charwomen and cleaners	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	753	Charwomen and cleaners	X	X	X	X	X	X	X	X	X	X 754	Cooks, except private household	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	754	Cooks, except private household	X	X	X	X	X	X	X	X	X	X 760	Counter and fountain workers	·	·	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	760	Counter and fountain workers	X	·	X	X	X	X	X	·	·	· 761	Elevator operators	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	761	Elevator operators	X	X	X	X	X	X	X	·	·	· 762	Firemen, fire protection	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	762	Firemen, fire protection	X	X	X	X	X	X	X	X	X	· 763	Guards, watchmen, and doorkeepers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	763	Guards, watchmen, and doorkeepers	X	X	X	X	X	X	X	X	X	X 764	Housekeepers and stewards, except private household	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	764	Housekeepers and stewards, except private household	X	X	X	X	X	X	X	X	X	X 770	Janitors and sextons	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	770	Janitors and sextons	X	X	X	X	X	X	X	X	X	X 771	Marshals and constables	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	771	Marshals and constables	X	X	X	X	X	X	X	X	X	X 772	Midwives	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	772	Midwives	X	·	X	X	X	X	X	X	X	· 773	Policemen and detectives	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	773	Policemen and detectives	X	X	X	X	X	X	X	X	X	X 780	Porters	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	780	Porters	X	X	X	X	X	X	X	X	X	X 781	Practical nurses	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	781	Practical nurses	X	X	X	X	X	X	X	X	X	X 782	Sheriffs and bailiffs	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	782	Sheriffs and bailiffs	X	X	X	X	X	X	X	X	X	X 783	Ushers, recreation and amusement	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	783	Ushers, recreation and amusement	X	X	X	X	X	X	X	·	X	· 784	Waiters and waitresses	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	784	Waiters and waitresses	X	X	X	X	X	X	X	X	X	X 785	Watchmen (crossing) and bridge tenders	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	785	Watchmen (crossing) and bridge tenders	X	X	X	X	X	X	X	X	X	X 790	Service workers, except private household (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	790	Service workers, except private household (n.e.c.)	X	X	X	X	X	X	X	X	X	X Farm Laborers:																											Farm Laborers: 810	Farm foremen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	810	Farm foremen	X	X	X	X	X	X	X	X	X	X 820	Farm laborers, wage workers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	820	Farm laborers, wage workers	X	X	X	X	X	X	X	X	X	X 830	Farm laborers, unpaid family workers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	830	Farm laborers, unpaid family workers	X	X	X	X	X	X	·	·	·	· 840	Farm service laborers, self-employed	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	840	Farm service laborers, self-employed	X	·	X	X	X	X	X	X	X	X Laborers:																											Laborers: 910	Fishermen and oystermen	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	910	Fishermen and oystermen	X	X	X	X	X	X	X	X	X	X Code	Label 2019 acs 2018 acs 2017 acs 2016 acs 2015 acs 2014 acs 2013 acs 2012 acs 2011 acs 2010 acs 2009 acs 2008 acs 2007 acs 2006 acs 2005 acs 2004 acs 2003 acs 2002 acs 2001 acs 2000 5pct 1990 5pct 1980 5pct 1970 met2 1970 met1 1960 5pct Code	Label 1950 1pct 1940 1pct 1930 5pct 1920 1pct 1910 1pct 1900 5pct 1880 10pct 1870 1pct 1860 1pct 1850 1%+ 920	Garage laborers and car washers and greasers	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	920	Garage laborers and car washers and greasers	X	X	X	X	X	X	·	·	·	· 930	Gardeners, except farm and groundskeepers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	930	Gardeners, except farm and groundskeepers	X	X	X	X	X	X	X	X	X	X 940	Longshoremen and stevedores	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	X	X	940	Longshoremen and stevedores	X	X	X	X	X	X	X	X	X	X 950	Lumbermen, raftsmen, and woodchoppers	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	950	Lumbermen, raftsmen, and woodchoppers	X	X	X	X	X	X	X	X	X	X 960	Teamsters	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	X	X	960	Teamsters	X	X	X	X	X	X	·	·	·	· 970	Laborers (n.e.c.)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	970	Laborers (n.e.c.)	X	X	X	X	X	X	X	X	X	X 979	Not yet classified	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	979	Not yet classified	·	·	·	·	·	·	·	·	·	· Non-occupational response:																											Non-occupational response: 980	Keeps house/housekeeping at home/housewife	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	980	Keeps house/housekeeping at home/housewife	·	·	X	X	X	X	X	X	X	· 981	Imputed keeping house (1850-1900)	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	981	Imputed keeping house (1850-1900)	·	·	·	·	·	X	X	X	X	· 982	Helping at home/helps parents/housework	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	982	Helping at home/helps parents/housework	·	·	X	X	X	X	X	X	X	· 983	At school/student	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	983	At school/student	·	·	X	X	X	X	X	X	X	X 984	Retired	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	984	Retired	·	·	X	X	X	X	X	X	X	X 985	Unemployed/without occupation	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	985	Unemployed/without occupation	·	·	·	·	·	·	X	·	·	· 986	Invalid/disabled w/ no occupation reported	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	986	Invalid/disabled w/ no occupation reported	·	·	X	X	X	X	X	X	X	X 987	Inmate	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	987	Inmate	·	·	X	X	X	X	X	X	X	X 990	New Worker	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	990	New Worker	·	·	·	·	·	·	·	·	·	· 991	Gentleman/lady/at leisure	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	991	Gentleman/lady/at leisure	·	·	X	·	·	·	·	·	·	· 995	Other non-occupational response	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	995	Other non-occupational response	·	·	X	X	X	X	X	X	X	X 997	Occupation missing/unknown	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	·	X	997	Occupation missing/unknown	X	X	·	·	·	·	·	·	·	· 999	N/A (blank)	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	X	999	N/A (blank)	X	X	X	X	X	X	X	·	X	X"
a = {}

state = None
for i in s.split():
    try: 
        int(i)
        state = i
        cat = ""
    except: 
        if i in ["X", "·"]:
            if state:
                a[state] = cat[1:]
                state = None
            else:
                continue
        else:
            cat = cat + " " + i

In [ ]:
t = "000	N/A (not applicable) 003	Legislators 004	Chief executives and general administrators, public administration 005	Administrators and officials, public administration 006	Administrators, protective services 007	Financial managers 008	Personnel and labor relations managers 009	Purchasing managers 013	Managers, marketing, advertising, and public relations 014	Administrators, education and related fields 015	Managers, medicine and health 016	Postmasters and mail superintendents 017	Managers, food serving and lodging establishments 018	Managers, properties and real estate 019	Funeral directors 021	Managers, service organizations, n.e.c. 022	Managers and administrators, n.e.c. Management Related Occupations: 023	Accountants and auditors 024	Underwriters 025	Other financial officers 026	Management analysts 027	Personnel, training, and labor relations specialists 028	Purchasing agents and buyers, farm products 029	Buyers, wholesale and retail trade, except farm products 033	Purchasing agents and buyers, n.e.c. 034	Business and promotion agents 035	Construction inspectors 036	Inspectors and compliance officers, except construction 037	Management related occupations, n.e.c. Professional Specialty Occupations Engineers, Architects, and Surveyors: 043	Architects Engineers: 044	Aerospace 045	Metallurgical and materials 046	Mining 047	Petroleum 048	Chemical 049	Nuclear 053	Civil 054	Agricultural 055	Electrical and electronic 056	Industrial 057	Mechanical 058	Marine and naval architects 059	Engineers, n.e.c. 063	Surveyors and mapping scientists Mathematical and Computer Scientists: 064	Computer systems analysts and scientists 065	Operations and systems researchers and analysts 066	Actuaries 067	Statisticians 068	Mathematical scientists, n.e.c. Natural Scientists: 069	Physicists and astronomers 073	Chemists, except biochemists 074	Atmospheric and space scientists 075	Geologists and geodesists 076	Physical scientists, n.e.c. 077	Agricultural and food scientists 078	Biological and life scientists 079	Forestry and conservation scientists 083	Medical scientists Health Diagnosing Occupations: 084	Physicians 085	Dentists 086	Veterinarians 087	Optometrists 088	Podiatrists 089	Health diagnosing practitioners, n.e.c. Health Assessment and Treating Occupations: 095	Registered nurses 096	Pharmacists 097	Dietitians Therapists: 098	Respiratory therapists 099	Occupational therapists 103	Physical therapists 104	Speech therapists 105	Therapists, n.e.c. 106	Physicians' assistants Teachers, Postsecondary: 113	Earth, environmental, and marine science teachers 114	Biological science teachers 115	Chemistry teachers 116	Physics teachers 117	Natural science teachers, n.e.c. 118	Psychology teachers 119	Economics teachers 123	History teachers 124	Political science teachers 125	Sociology teachers 126	Social science teachers, n.e.c. 127	Engineering teachers 128	Mathematical science teachers 129	Computer science teachers 133	Medical science teachers 134	Health specialties teachers 135	Business, commerce, and marketing teachers 136	Agriculture and forestry teachers 137	Art, drama, and music teachers 138	Physical education teachers 139	Education teachers 143	English teachers 144	Foreign language teachers 145	Law teachers 146	Social work teachers 147	Theology teachers 148	Trade and industrial teachers 149	Home economics teachers 153	Teachers, postsecondary, n.e.c. 154	Postsecondary teachers, subject n.s. Teachers, Except Postsecondary: 155	Teachers, prekindergarten and kindergarten 156	Teachers, elementary school 157	Teachers, secondary school 158	Teachers, special education 159	Teachers, n.e.c. 163	Counselors, educational and vocational Librarians, Archivists, and Curators: 164	Librarians 165	Archivists and curators Social Scientists and Urban Planners: 166	Economists 167	Psychologists 168	Sociologists 169	Social scientists, n.e.c. 173	Urban planners Social, Recreation, and Religious Workers 174	Social workers 175	Recreation workers 176	Clergy 177	Religious workers, n.e.c. Lawyers and Judges: 178	Lawyers 179	Judges Writers, Artists, Entertainers, and Athletes: 183	Authors 184	Technical writers 185	Designers 186	Musicians and composers 187	Actors and directors 188	Painters, sculptors, craft-artists, and artist printmakers 189	Photographers 193	Dancers 194	Artists, performers, and related workers, n.e.c. 195	Editors and reporters 197	Public relations specialists 198	Announcers 199	Athletes Technical Sales and Administrative Support Occupations Technicians and Related Support Occupations - Health Technologists and Technicians: 203	Clinical laboratory technologists and technicians 204	Dental hygienists 205	Health record technologists and technicians 206	Radiologic technicians 207	Licensed practical nurses 208	Health technologists and technicians, n.e.c. Technologists and Technicians, Except Health - Engineering and Related Technologists and Technicians: 213	Electrical and electronic technicians 214	Industrial engineering technicians 215	Mechanical engineering technicians 216	Engineering technicians, n.e.c. 217	Drafting occupations 218	Surveying and mapping technicians - Science Technicians: 223	Biological technicians 224	Chemical technicians 225	Science technicians n.e.c. - Technicians, Except Health, Engineering, and Science: 226	Airplane pilots and navigators 227	Air traffic controllers 228	Broadcast equipment operators 229	Computer programmers 233	Tool programmers, numerical control 234	Legal assistants 235	Technician, n.e.c. Sales Occupations 243	Supervisors and proprietors, sales occupations - Sales Representatives, Finance and Business Services: 253	Insurance sales occupations 254	Real estate sales occupations 255	Securities and financial services sales occupations 256	Advertising and related sales occupations 257	Sales occupations, other business services - Sales Representatives, Commodities Except Retail: 258	Sales engineers 259	Sales representatives, mining, manufacturing, and wholesale - Sales Workers, Retail and Personal Services: 263	Sales workers, motor vehicles and boats 264	Sales workers, apparel 265	Sales workers, shoes 266	Sales workers, furniture and home furnishings 267	Sales workers, radio, TV, hi-fi, and appliances 268	Sales workers, hardware and building supplies 269	Sales workers, parts 274	Sales workers, other commodities 275	Sales counter clerks 276	Cashiers 277	Street and door-to-door sales workers 278	News vendors - Sales Related Occupations: 283	Demonstrators, promoters and models, sales 284	Auctioneers 285	Sales support occupations, n.e.c. 290	Sales workers -- allocated Administrative Support Occupations, Including Clerical - Supervisors, Administrative Support Occupations: 303	Supervisors, general office 304	Supervisors, computer equipment operators 305	Supervisors, financial records processing 306	Chief communications operators 307	Supervisors; distribution, scheduling, and adjusting clerks - Computer Equipment Operators: 308	Computer operators 309	Peripheral equipment operators - Secretaries, Stenographers, and Typists: 313	Secretaries 314	Stenographers 315	Typists - Information Clerks: 316	Interviewers 317	Hotel clerks 318	Transportation ticket and reservation agents 319	Receptionists 323	Information clerks, n.e.c. - Records Processing Occupations, Except Financial: 325	Classified-ad clerks 326	Correspondence clerks 327	Order clerks 328	Personnel clerks, except payroll and timekeeping 329	Library clerks 335	File clerks 336	Records clerks - Financial Records Processing Occupations: 337	Bookkeepers, accounting, and auditing clerks 338	Payroll and timekeeping clerks 339	Billing clerks 343	Cost and rate clerks 344	Billing, posting, and calculating machine operators - Duplicating, Mail, and Other Office Machine Operators: 345	Duplicating machine operators 346	Mail preparing and paper handling machine operators 347	Office machine operators, n.e.c. - Communications Equipment Operators: 348	Telephone operators 353	Communications equipment operators, n.e.c. - Mail and Message Distributing Occupations: 354	Postal clerks except mail carriers 355	Mail carriers, postal service 356	Mail clerks, except postal service 357	Messengers - Material Recording, Scheduling, and Distributing Clerks: 359	Dispatchers 363	Production coordinators 364	Traffic, shipping, and receiving clerks 365	Stock and inventory clerks 366	Meter readers 368	Weighers, measurers, checkers, and samplers 373	Expediters 374	Material recording, scheduling, and distributing clerks, n.e.c. - Adjusters and Investigators: 375	Insurance adjusters, examiners, and investigators 376	Investigators and adjusters, except insurance 377	Eligibility clerks, social welfare 378	Bill and account collectors - Miscellaneous Administrative Support Occupations: 379	General office clerks 383	Bank tellers 384	Proofreaders 385	Data-entry keyers 386	Statistical clerks 387	Teachers' aides 389	Administrative support occupations, n.e.c. 390	Professional, technical, and kindred workers -- allocated 391	Clerical and kindred workers -- allocated Service Occupations Private Household Occupations 403	Launderers and ironers 404	Cooks, private household 405	Housekeepers and butlers 406	Child care workers, private household 407	Private household cleaners and servants 408	Private household workers -- allocated Protective Service Occupations - Supervisors, Protective Service Occupations: 413	Supervisors, firefighting and fire prevention occupations 414	Supervisors, police and detectives 415	Supervisors, guards - Firefighting and Fire Prevention Occupations: 416	Fire inspection and fire prevention occupations 417	Firefighting occupations - Police and Detectives: 418	Police and detectives, public service 423	Sheriffs, bailiffs, and other law enforcement officers 424	Correctional institution officers - Guards: 425	Crossing guards 426	Guards and police, except public service 427	Protective service occupations, n.e.c. Service Occupations, Except Protective and Household - Food Preparation and Service Occupations: 433	Supervisors, food preparation and service occupations 434	Bartenders 435	Waiters and waitresses 436	Cooks 438	Food counter, fountain and related occupations 439	Kitchen workers, food preparation 443	Waiters'/waitresses' assistants 444	Miscellaneous food preparation occupations - Health Service Occupations: 445	Dental assistants 446	Health aides, except nursing 447	Nursing aides, orderlies, and attendants - Cleaning and Building Service Occupations, Except Household: 448	Supervisors, cleaning and building service workers 449	Maids and housemen 453	Janitors and cleaners 454	Elevator operators 455	Pest control occupations - Personal Service Occupations: 456	Supervisors, personal service occupations 457	Barbers 458	Hairdressers and cosmetologists 459	Attendants, amusement and recreation facilities 461	Guides 462	Ushers 463	Public transportation attendants 464	Baggage porters and bellhops 465	Welfare service aides 466	Family child care providers 467	Early childhood teachers' assistants 468	Child care workers, n.e.c. 469	Personal service occupations, n.e.c. Farming, Forestry, and Fishing Occupations Farm Operators and Managers: 473	Farmers, except horticultural 474	Horticultural specialty farmers 475	Managers, farms, except horticultural 476	Managers, horticultural specialty farms Other Agricultural and Related Occupations - Farm Occupations, Except Managerial: 477	Supervisors, farm workers 479	Farm workers 480	Farm laborers and farm foremen -- allocated 483	Marine life cultivation workers 484	Nursery workers - Related Agricultural Occupations: 485	Supervisors, related agricultural occupations 486	Groundskeepers and gardeners, except farm 487	Animal caretakers, except farm 488	Graders and sorters, agricultural products 489	Inspectors, agricultural products - Forestry and Logging Occupations: 494	Supervisors, forestry, and logging workers 495	Forestry workers except logging 496	Timber cutting and logging occupations - Fishers, Hunters, and Trappers: 497	Captains and other officers, fishing vessels 498	Fishers 499	Hunters and trappers PRECISION PRODUCTION, CRAFT, AND REPAIR OCCUPATIONS Mechanics and Repairers 503	Supervisors, mechanics and repairers Mechanics and Repairers, Except Supervisors - Vehicle and Mobile Equipment Mechanics and Repairers: 505	Automobile mechanics 506	Automobile mechanic apprentices 507	Bus, truck, and stationary engine mechanics 508	Aircraft engine mechanics 509	Small engine repairers 514	Automobile body and related repairers 515	Aircraft mechanics, except engine 516	Heavy equipment mechanics 517	Farm equipment mechanics 518	Industrial machinery repairers 519	Machinery maintenance occupations - Electrical and Electronic Equipment Repairers: 523	Electronic repairers, communications and industrial equipment 525	Data processing equipment repairers 526	Household appliance and power tool repairers 527	Telephone line installers and repairers 529	Telephone installers and repairers 533	Misc. electrical and electronic equipment repairers 534	Heating, air conditioning, and refrigeration mechanics - Miscellaneous Mechanics and Repairers: 535	Camera, watch, and musical instrument repairers 536	Locksmiths and safe repairers 538	Office machine repairers 539	Mechanical controls and valve repairers 543	Elevator installers and repairers 544	Millwrights 547	Mechanics and repairers, n.e.c. 549	Mechanics and repairers, n.s. Construction Trades - Supervisors, Construction Occupations: 553	Supervisors, brickmasons, stonemasons, and tile setters 554	Supervisors, carpenters and related workers 555	Supervisors, electricians and power transmission installers 556	Supervisors, painters, paperhangers, and plasterers 557	Supervisors, plumbers, pipefitters, and steamfitters 558	Supervisors, construction, n.e.c. - Construction Trades, Except Supervisors: 563	Brickmasons and stonemasons 564	Brickmason and stonemason apprentices 565	Tile setters, hard and soft 566	Carpet installers 567	Carpenters 569	Carpenter apprentices 573	Drywall installers 575	Electricians 576	Electrician apprentices 577	Electrical power installers and repairers 579	Painters, construction and maintenance 583	Paperhangers 584	Plasterers 585	Plumbers, pipefitters, and steamfitters 587	Plumber, pipe fitter, and steamfitter apprentices 588	Concrete and terrazzo finishers 589	Glaziers 593	Insulation workers 594	Paving, surfacing, and tamping equipment operators 595	Roofers 596	Sheetmetal duct installers 597	Structural metal workers 598	Drillers, earth 599	Construction trades, n.e.c. Extractive Occupations 613	Supervisors, extractive occupations 614	Drillers, oil well 615	Explosives workers 616	Mining machine operators 617	Mining occupations, n.e.c. Precision Production Occupations 628	Supervisors, production occupations - Precision Metal Working Occupations: 634	Tool and die makers 635	Tool and die maker apprentices 636	Precision assemblers, metal 637	Machinists 639	Machinist apprentices 643	Boilermakers 644	Precision grinders, filers, and tool sharpeners 645	Patternmakers and model makers, metal 646	Lay-out workers 647	Precious stones and metals workers (jewelers) 649	Engravers, metal 653	Sheet metal workers 654	Sheet metal worker apprentices 655	Miscellaneous precision metal workers - Precision Woodworking Occupations: 656	Patternmakers and model makers, wood 657	Cabinet makers and bench carpenters 658	Furniture and wood finishers 659	Miscellaneous precision woodworkers - Precision Textile, Apparel, and Furnishings Machine Workers: 666	Dressmakers 667	Tailors 668	Upholsterers 669	Shoe repairers 674	Miscellaneous precision apparel and fabric workers - Precision Workers, Assorted Materials: 675	Hand molders and shapers, except jewelers 676	Patternmakers, lay-out workers, and cutters 677	Optical goods workers 678	Dental laboratory and medical appliance technicians 679	Bookbinders 683	Electrical and electronic equipment assemblers 684	Miscellaneous precision workers, n.e.c. - Precision Food Production Occupations: 686	Butchers and meat cutters 687	Bakers 688	Food batchmakers - Precision Inspectors, Testers, and Related Workers: 689	Inspectors, testers, and graders 693	Adjusters and calibrators - Plant and System Operators: 694	Water and sewage treatment plant operators 695	Power plant operators 696	Stationary engineers 699	Miscellaneous plant and system operators Operators, Fabricators, and Laborers Machine Operators, Assemblers, and Inspectors Machine Operators and Tenders, Except Precision - Metalworking and Plastic Working Machine Operators: 703	Lathe and turning machine set-up operators 704	Lathe and turning machine operators 705	Milling and planing machine operators 706	Punching and stamping press machine operators 707	Rolling machine operators 708	Drilling and boring machine operators 709	Grinding, abrading, buffing, and polishing machine operators 713	Forging machine operators 714	Numerical control machine operators 715	Miscellaneous metal, plastic, stone, and glass working machine operators 717	Fabricating machine operators, n.e.c. - Metal and Plastic Processing Machine Operators: 719	Molding and casting machine operators 723	Metal plating machine operators 724	Heat treating equipment operators 725	Miscellaneous metal and plastic processing machine operators - Woodworking Machine Operators: 726	Wood lathe, routing, and planing machine operators 727	Sawing machine operators 728	Shaping and joining machine operators 729	Nailing and tacking machine operators 733	Miscellaneous woodworking machine operators - Printing Machine Operators: 734	Printing press operators 735	Photoengravers and lithographers 736	Typesetters and compositors 737	Miscellaneous printing machine operators - Textile, Apparel, and Furnishings Machine Operators: 738	Winding and twisting machine operators 739	Knitting, looping, taping, and weaving machine operators 743	Textile cutting machine operators 744	Textile sewing machine operators 745	Shoe machine operators 747	Pressing machine operators 748	Laundering and dry cleaning machine operators 749	Miscellaneous textile machine operators - Machine Operators, Assorted Materials: 753	Cementing and gluing machine operators 754	Packaging and filling machine operators 755	Extruding and forming machine operators 756	Mixing and blending machine operators 757	Separating, filtering, and clarifying machine operators 758	Compressing and compacting machine operators 759	Painting and paint spraying machine operators 763	Roasting and baking machine operators, food 764	Washing, cleaning, and pickling machine operators 765	Folding machine operators 766	Furnace, kiln, and oven operators, except food 768	Crushing and grinding machine operators 769	Slicing and cutting machine operators 773	Motion picture projectionists 774	Photographic process machine operators 777	Miscellaneous machine operators, n.e.c. 779	Machine operators, n.s. Fabricators, Assemblers, and Hand Working Occupations 783	Welders and cutters 784	Solderers and brazers 785	Assemblers 786	Hand cutting and trimming occupations 787	Hand molding, casting, and forming occupations 789	Hand painting, coating, and decorating occupations 793	Hand engraving and printing occupations 795	Miscellaneous hand working occupations Production Inspectors, Testers, Samplers, and Weighers 796	Production inspectors, checkers, and examiners 797	Production testers 798	Production samplers and weighers 799	Graders and sorters, except agricultural Transportation and Material Moving Occupations - Motor Vehicle Operators: 803	Supervisors, motor vehicle operators 804	Truck drivers 806	Driver-sales workers 808	Bus drivers 809	Taxicab drivers and chauffeurs 813	Parking lot attendants 814	Motor transportation occupations, n.e.c. 815	Transport equipment operatives -- allocated Transportation Occupations, Except Motor Vehicles - Rail Transportation Occupations: 823	Railroad conductors and yardmasters 824	Locomotive operating occupations 825	Railroad brake, signal, and switch operators 826	Rail vehicle operators, n.e.c. - Water Transportation Occupations: 828	Ship captains and mates, except fishing boats 829	Sailors and deckhands 833	Marine engineers 834	Bridge, lock, and lighthouse tenders - Material Moving Equipment Operators: 843	Supervisors, material moving equipment operators 844	Operating engineers 845	Longshore equipment operators 848	Hoist and winch operators 849	Crane and tower operators 853	Excavating and loading machine operators 855	Grader, dozer, and scraper operators 856	Industrial truck and tractor equipment operators 859	Miscellaneous material moving equipment operators 864	Supervisors, handlers, equipment cleaners, and laborers, n.e.c. 865	Helpers, mechanics and repairers - Helpers, Construction and Extractive Occupations: 866	Helpers, construction trades 867	Helpers, surveyor 868	Helpers, extractive occupations 869	Construction laborers 874	Production helpers - Freight, Stock, and Material Handlers: 875	Garbage collectors 876	Stevedores 877	Stock handlers and baggers 878	Machine feeders and offbearers 883	Freight, stock, and material handlers, n.e.c. 885	Garage and service station related occupations 887	Vehicle washers and equipment cleaners 888	Hand packers and packagers 889	Laborers, except construction 890	Laborers, except farm -- allocated Military Occupations 903	Commissioned Officers and Warrant Officers 904	Non-commissioned Officers and Other Enlisted Personnel 905	Military occupation, rank not specified Experienced Unemployed Not Classified by Occupation 909	Last worked 1984 or earlier"
no = 0
k = 0
w = False
b = {}
d = False
for i in t.split():
    try: 
        int(i)
        if no == 0: 
            no = i
            cat = ""
            no = 1
        cat = cat[1:]
        if cat == "":
            print("here")
            continue
        for j, c  in enumerate(cat):
            if c == "-":
                w = True
            if c.isupper():
                if d == True:
                    if w == True:
                        cat = cat[:j-3]
                        d = False
                        w= False
                        break
                    cat = cat[:j]
                    d = False
                    w= False
                    break
                else:
                    d = True
                    
                    
        b[no] = cat
        no = i
        cat = ""
        d= False
        w= False
    except: 
        if i[-1] == ":":
            continue
        else:
            cat = cat + " " + i
        

In [3]:
occ1990_map = {"000": 'N/A (not applicable)',
 '003': 'Legislators',
 '004': 'Chief executives and general administrators, public administration',
 '005': 'Administrators and officials, public administration',
 '006': 'Administrators, protective services',
 '007': 'Financial managers',
 '008': 'Personnel and labor relations managers',
 '009': 'Purchasing managers',
 '013': 'Managers, marketing, advertising, and public relations',
 '014': 'Administrators, education and related fields',
 '015': 'Managers, medicine and health',
 '016': 'Postmasters and mail superintendents',
 '017': 'Managers, food serving and lodging establishments',
 '018': 'Managers, properties and real estate',
 '019': 'Funeral directors',
 '021': 'Managers, service organizations, n.e.c.',
 '022': 'Managers and administrators, n.e.c. ',
 '023': 'Accountants and auditors',
 '024': 'Underwriters',
 '025': 'Other financial officers',
 '026': 'Management analysts',
 '027': 'Personnel, training, and labor relations specialists',
 '028': 'Purchasing agents and buyers, farm products',
 '029': 'Buyers, wholesale and retail trade, except farm products',
 '033': 'Purchasing agents and buyers, n.e.c.',
 '034': 'Business and promotion agents',
 '035': 'Construction inspectors',
 '036': 'Inspectors and compliance officers, except construction',
 '037': 'Management related occupations, n.e.c. ',
 '043': 'Architects',
 '044': 'Aerospace',
 '045': 'Metallurgical and materials',
 '046': 'Mining',
 '047': 'Petroleum',
 '048': 'Chemical',
 '049': 'Nuclear',
 '053': 'Civil',
 '054': 'Agricultural',
 '055': 'Electrical and electronic',
 '056': 'Industrial',
 '057': 'Mechanical',
 '058': 'Marine and naval architects',
 '059': 'Engineers, n.e.c.',
 '063': 'Surveyors and mapping scientists ',
 '064': 'Computer systems analysts and scientists',
 '065': 'Operations and systems researchers and analysts',
 '066': 'Actuaries',
 '067': 'Statisticians',
 '068': 'Mathematical scientists, n.e.c. ',
 '069': 'Physicists and astronomers',
 '073': 'Chemists, except biochemists',
 '074': 'Atmospheric and space scientists',
 '075': 'Geologists and geodesists',
 '076': 'Physical scientists, n.e.c.',
 '077': 'Agricultural and food scientists',
 '078': 'Biological and life scientists',
 '079': 'Forestry and conservation scientists',
 '083': 'Medical scientists ',
 '084': 'Physicians',
 '085': 'Dentists',
 '086': 'Veterinarians',
 '087': 'Optometrists',
 '088': 'Podiatrists',
 '089': 'Health diagnosing practitioners, n.e.c. ',
 '095': 'Registered nurses',
 '096': 'Pharmacists',
 '097': 'Dietitians',
 '098': 'Respiratory therapists',
 '099': 'Occupational therapists',
 '103': 'Physical therapists',
 '104': 'Speech therapists',
 '105': 'Therapists, n.e.c.',
 '106': "Physicians' assistants ",
 '113': 'Earth, environmental, and marine science teachers',
 '114': 'Biological science teachers',
 '115': 'Chemistry teachers',
 '116': 'Physics teachers',
 '117': 'Natural science teachers, n.e.c.',
 '118': 'Psychology teachers',
 '119': 'Economics teachers',
 '123': 'History teachers',
 '124': 'Political science teachers',
 '125': 'Sociology teachers',
 '126': 'Social science teachers, n.e.c.',
 '127': 'Engineering teachers',
 '128': 'Mathematical science teachers',
 '129': 'Computer science teachers',
 '133': 'Medical science teachers',
 '134': 'Health specialties teachers',
 '135': 'Business, commerce, and marketing teachers',
 '136': 'Agriculture and forestry teachers',
 '137': 'Art, drama, and music teachers',
 '138': 'Physical education teachers',
 '139': 'Education teachers',
 '143': 'English teachers',
 '144': 'Foreign language teachers',
 '145': 'Law teachers',
 '146': 'Social work teachers',
 '147': 'Theology teachers',
 '148': 'Trade and industrial teachers',
 '149': 'Home economics teachers',
 '153': 'Teachers, postsecondary, n.e.c.',
 '154': 'Postsecondary teachers, subject n.s. ',
 '155': 'Teachers, prekindergarten and kindergarten',
 '156': 'Teachers, elementary school',
 '157': 'Teachers, secondary school',
 '158': 'Teachers, special education',
 '159': 'Teachers, n.e.c.',
 '163': 'Counselors, educational and vocational ',
 '164': 'Librarians',
 '165': 'Archivists and curators ',
 '166': 'Economists',
 '167': 'Psychologists',
 '168': 'Sociologists',
 '169': 'Social scientists, n.e.c.',
 '173': 'Urban planners ',
 '174': 'Social workers',
 '175': 'Recreation workers',
 '176': 'Clergy',
 '177': 'Religious workers, n.e.c. ',
 '178': 'Lawyers',
 '179': 'Judges ',
 '183': 'Authors',
 '184': 'Technical writers',
 '185': 'Designers',
 '186': 'Musicians and composers',
 '187': 'Actors and directors',
 '188': 'Painters, sculptors, craft-artists, and artist printmakers',
 '189': 'Photographers',
 '193': 'Dancers',
 '194': 'Artists, performers, and related workers, n.e.c.',
 '195': 'Editors and reporters',
 '197': 'Public relations specialists',
 '198': 'Announcers',
 '199': 'Athletes ',
 '203': 'Clinical laboratory technologists and technicians',
 '204': 'Dental hygienists',
 '205': 'Health record technologists and technicians',
 '206': 'Radiologic technicians',
 '207': 'Licensed practical nurses',
 '208': 'Health technologists and technicians, n.e.c. ',
 '213': 'Electrical and electronic technicians',
 '214': 'Industrial engineering technicians',
 '215': 'Mechanical engineering technicians',
 '216': 'Engineering technicians, n.e.c.',
 '217': 'Drafting occupations',
 '218': 'Surveying and mapping technicians',
 '223': 'Biological technicians',
 '224': 'Chemical technicians',
 '225': 'Science technicians n.e.c.',
 '226': 'Airplane pilots and navigators',
 '227': 'Air traffic controllers',
 '228': 'Broadcast equipment operators',
 '229': 'Computer programmers',
 '233': 'Tool programmers, numerical control',
 '234': 'Legal assistants',
 '235': 'Technician, n.e.c. ',
 '243': 'Supervisors and proprietors, sales occupations',
 '253': 'Insurance sales occupations',
 '254': 'Real estate sales occupations',
 '255': 'Securities and financial services sales occupations',
 '256': 'Advertising and related sales occupations',
 '257': 'Sales occupations, other business services',
 '258': 'Sales engineers',
 '259': 'Sales representatives, mining, manufacturing, and wholesale',
 '263': 'Sales workers, motor vehicles and boats',
 '264': 'Sales workers, apparel',
 '265': 'Sales workers, shoes',
 '266': 'Sales workers, furniture and home furnishings',
 '267': 'Sales workers, radio, ',
 '268': 'Sales workers, hardware and building supplies',
 '269': 'Sales workers, parts',
 '274': 'Sales workers, other commodities',
 '275': 'Sales counter clerks',
 '276': 'Cashiers',
 '277': 'Street and door-to-door sales workers',
 '278': 'News vendors',
 '283': 'Demonstrators, promoters and models, sales',
 '284': 'Auctioneers',
 '285': 'Sales support occupations, n.e.c.',
 '290': 'Sales workers -- allocat',
 '303': 'Supervisors, general office',
 '304': 'Supervisors, computer equipment operators',
 '305': 'Supervisors, financial records processing',
 '306': 'Chief communications operators',
 '307': 'Supervisors; distribution, scheduling, and adjusting clerks',
 '308': 'Computer operators',
 '309': 'Peripheral equipment operators',
 '313': 'Secretaries',
 '314': 'Stenographers',
 '315': 'Typists',
 '316': 'Interviewers',
 '317': 'Hotel clerks',
 '318': 'Transportation ticket and reservation agents',
 '319': 'Receptionists',
 '323': 'Information clerks, n.e.c.',
 '325': 'Classified-ad clerks',
 '326': 'Correspondence clerks',
 '327': 'Order clerks',
 '328': 'Personnel clerks, except payroll and timekeeping',
 '329': 'Library clerks',
 '335': 'File clerks',
 '336': 'Records clerks',
 '337': 'Bookkeepers, accounting, and auditing clerks',
 '338': 'Payroll and timekeeping clerks',
 '339': 'Billing clerks',
 '343': 'Cost and rate clerks',
 '344': 'Billing, posting, and calculating machine operators',
 '345': 'Duplicating machine operators',
 '346': 'Mail preparing and paper handling machine operators',
 '347': 'Office machine operators, n.e.c.',
 '348': 'Telephone operators',
 '353': 'Communications equipment operators, n.e.c.',
 '354': 'Postal clerks except mail carriers',
 '355': 'Mail carriers, postal service',
 '356': 'Mail clerks, except postal service',
 '357': 'Messengers',
 '359': 'Dispatchers',
 '363': 'Production coordinators',
 '364': 'Traffic, shipping, and receiving clerks',
 '365': 'Stock and inventory clerks',
 '366': 'Meter readers',
 '368': 'Weighers, measurers, checkers, and samplers',
 '373': 'Expediters',
 '374': 'Material recording, scheduling, and distributing clerks, n.e.c.',
 '375': 'Insurance adjusters, examiners, and investigators',
 '376': 'Investigators and adjusters, except insurance',
 '377': 'Eligibility clerks, social welfare',
 '378': 'Bill and account collectors',
 '379': 'General office clerks',
 '383': 'Bank tellers',
 '384': 'Proofreaders',
 '385': 'Data-entry keyers',
 '386': 'Statistical clerks',
 '387': "Teachers' aides",
 '389': 'Administrative support occupations, n.e.c.',
 '390': 'Professional, technical, and kindred workers -- allocated',
 '391': 'Clerical and kindred workers -- allocat',
 '403': 'Launderers and ironers',
 '404': 'Cooks, private household',
 '405': 'Housekeepers and butlers',
 '406': 'Child care workers, private household',
 '407': 'Private household cleaners and servants',
 '408': 'Private household workers -- allocat',
 '413': 'Supervisors, firefighting and fire prevention occupations',
 '414': 'Supervisors, police and detectives',
 '415': 'Supervisors, guards',
 '416': 'Fire inspection and fire prevention occupations',
 '417': 'Firefighting occupations',
 '418': 'Police and detectives, public service',
 '423': 'Sheriffs, bailiffs, and other law enforcement officers',
 '424': 'Correctional institution officers -',
 '425': 'Crossing guards',
 '426': 'Guards and police, except public service',
 '427': 'Protective service occupations, n.e.c. ',
 '433': 'Supervisors, food preparation and service occupations',
 '434': 'Bartenders',
 '435': 'Waiters and waitresses',
 '436': 'Cooks',
 '438': 'Food counter, fountain and related occupations',
 '439': 'Kitchen workers, food preparation',
 '443': "Waiters'/waitresses' assistants",
 '444': 'Miscellaneous food preparation occupations',
 '445': 'Dental assistants',
 '446': 'Health aides, except nursing',
 '447': 'Nursing aides, orderlies, and attendants',
 '448': 'Supervisors, cleaning and building service workers',
 '449': 'Maids and housemen',
 '453': 'Janitors and cleaners',
 '454': 'Elevator operators',
 '455': 'Pest control occupations',
 '456': 'Supervisors, personal service occupations',
 '457': 'Barbers',
 '458': 'Hairdressers and cosmetologists',
 '459': 'Attendants, amusement and recreation facilities',
 '461': 'Guides',
 '462': 'Ushers',
 '463': 'Public transportation attendants',
 '464': 'Baggage porters and bellhops',
 '465': 'Welfare service aides',
 '466': 'Family child care providers',
 '467': "Early childhood teachers' assistants",
 '468': 'Child care workers, n.e.c.',
 '469': 'Personal service occupations, n.e.c. ',
 '473': 'Farmers, except horticultural',
 '474': 'Horticultural specialty farmers',
 '475': 'Managers, farms, except horticultural',
 '476': 'Managers, horticultural specialty farms ',
 '477': 'Supervisors, farm workers',
 '479': 'Farm workers',
 '480': 'Farm laborers and farm foremen -- allocated',
 '483': 'Marine life cultivation workers',
 '484': 'Nursery workers',
 '485': 'Supervisors, related agricultural occupations',
 '486': 'Groundskeepers and gardeners, except farm',
 '487': 'Animal caretakers, except farm',
 '488': 'Graders and sorters, agricultural products',
 '489': 'Inspectors, agricultural products',
 '494': 'Supervisors, forestry, and logging workers',
 '495': 'Forestry workers except logging',
 '496': 'Timber cutting and logging occupations',
 '497': 'Captains and other officers, fishing vessels',
 '498': 'Fishers',
 '499': 'Hunters and trappers ',
 '503': 'Supervisors, mechanics and repairers ',
 '505': 'Automobile mechanics',
 '506': 'Automobile mechanic apprentices',
 '507': 'Bus, truck, and stationary engine mechanics',
 '508': 'Aircraft engine mechanics',
 '509': 'Small engine repairers',
 '514': 'Automobile body and related repairers',
 '515': 'Aircraft mechanics, except engine',
 '516': 'Heavy equipment mechanics',
 '517': 'Farm equipment mechanics',
 '518': 'Industrial machinery repairers',
 '519': 'Machinery maintenance occupations',
 '523': 'Electronic repairers, communications and industrial equipment',
 '525': 'Data processing equipment repairers',
 '526': 'Household appliance and power tool repairers',
 '527': 'Telephone line installers and repairers',
 '529': 'Telephone installers and repairers',
 '533': 'Misc. electrical and electronic equipment repairers',
 '534': 'Heating, air conditioning, and refrigeration mechanics',
 '535': 'Camera, watch, and musical instrument repairers',
 '536': 'Locksmiths and safe repairers',
 '538': 'Office machine repairers',
 '539': 'Mechanical controls and valve repairers',
 '543': 'Elevator installers and repairers',
 '544': 'Millwrights',
 '547': 'Mechanics and repairers, n.e.c.',
 '549': 'Mechanics and repairers, n.s. ',
 '553': 'Supervisors, brickmasons, stonemasons, and tile setters',
 '554': 'Supervisors, carpenters and related workers',
 '555': 'Supervisors, electricians and power transmission installers',
 '556': 'Supervisors, painters, paperhangers, and plasterers',
 '557': 'Supervisors, plumbers, pipefitters, and steamfitters',
 '558': 'Supervisors, construction, n.e.c.',
 '563': 'Brickmasons and stonemasons',
 '564': 'Brickmason and stonemason apprentices',
 '565': 'Tile setters, hard and soft',
 '566': 'Carpet installers',
 '567': 'Carpenters',
 '569': 'Carpenter apprentices',
 '573': 'Drywall installers',
 '575': 'Electricians',
 '576': 'Electrician apprentices',
 '577': 'Electrical power installers and repairers',
 '579': 'Painters, construction and maintenance',
 '583': 'Paperhangers',
 '584': 'Plasterers',
 '585': 'Plumbers, pipefitters, and steamfitters',
 '587': 'Plumber, pipe fitter, and steamfitter apprentices',
 '588': 'Concrete and terrazzo finishers',
 '589': 'Glaziers',
 '593': 'Insulation workers',
 '594': 'Paving, surfacing, and tamping equipment operators',
 '595': 'Roofers',
 '596': 'Sheetmetal duct installers',
 '597': 'Structural metal workers',
 '598': 'Drillers, earth',
 '599': 'Construction trades, n.e.c. ',
 '613': 'Supervisors, extractive occupations',
 '614': 'Drillers, oil well',
 '615': 'Explosives workers',
 '616': 'Mining machine operators',
 '617': 'Mining occupations, n.e.c. ',
 '628': 'Supervisors, production occupations',
 '634': 'Tool and die makers',
 '635': 'Tool and die maker apprentices',
 '636': 'Precision assemblers, metal',
 '637': 'Machinists',
 '639': 'Machinist apprentices',
 '643': 'Boilermakers',
 '644': 'Precision grinders, filers, and tool sharpeners',
 '645': 'Patternmakers and model makers, metal',
 '646': 'Lay-out workers',
 '647': 'Precious stones and metals workers (jewelers)',
 '649': 'Engravers, metal',
 '653': 'Sheet metal workers',
 '654': 'Sheet metal worker apprentices',
 '655': 'Miscellaneous precision metal workers',
 '656': 'Patternmakers and model makers, wood',
 '657': 'Cabinet makers and bench carpenters',
 '658': 'Furniture and wood finishers',
 '659': 'Miscellaneous precision woodworkers',
 '666': 'Dressmakers',
 '667': 'Tailors',
 '668': 'Upholsterers',
 '669': 'Shoe repairers',
 '674': 'Miscellaneous precision apparel and fabric workers',
 '675': 'Hand molders and shapers, except jewelers',
 '676': 'Patternmakers, lay-out workers, and cutters',
 '677': 'Optical goods workers',
 '678': 'Dental laboratory and medical appliance technicians',
 '679': 'Bookbinders',
 '683': 'Electrical and electronic equipment assemblers',
 '684': 'Miscellaneous precision workers, n.e.c.',
 '686': 'Butchers and meat cutters',
 '687': 'Bakers',
 '688': 'Food batchmakers',
 '689': 'Inspectors, testers, and graders',
 '693': 'Adjusters and calibrators',
 '694': 'Water and sewage treatment plant operators',
 '695': 'Power plant operators',
 '696': 'Stationary engineers',
 '699': 'Miscellaneous plant and system operators ',
 '703': 'Lathe and turning machine set-up operators',
 '704': 'Lathe and turning machine operators',
 '705': 'Milling and planing machine operators',
 '706': 'Punching and stamping press machine operators',
 '707': 'Rolling machine operators',
 '708': 'Drilling and boring machine operators',
 '709': 'Grinding, abrading, buffing, and polishing machine operators',
 '713': 'Forging machine operators',
 '714': 'Numerical control machine operators',
 '715': 'Miscellaneous metal, plastic, stone, and glass working machine operators',
 '717': 'Fabricating machine operators, n.e.c.',
 '719': 'Molding and casting machine operators',
 '723': 'Metal plating machine operators',
 '724': 'Heat treating equipment operators',
 '725': 'Miscellaneous metal and plastic processing machine operators',
 '726': 'Wood lathe, routing, and planing machine operators',
 '727': 'Sawing machine operators',
 '728': 'Shaping and joining machine operators',
 '729': 'Nailing and tacking machine operators',
 '733': 'Miscellaneous woodworking machine operators',
 '734': 'Printing press operators',
 '735': 'Photoengravers and lithographers',
 '736': 'Typesetters and compositors',
 '737': 'Miscellaneous printing machine operators',
 '738': 'Winding and twisting machine operators',
 '739': 'Knitting, looping, taping, and weaving machine operators',
 '743': 'Textile cutting machine operators',
 '744': 'Textile sewing machine operators',
 '745': 'Shoe machine operators',
 '747': 'Pressing machine operators',
 '748': 'Laundering and dry cleaning machine operators',
 '749': 'Miscellaneous textile machine operators',
 '753': 'Cementing and gluing machine operators',
 '754': 'Packaging and filling machine operators',
 '755': 'Extruding and forming machine operators',
 '756': 'Mixing and blending machine operators',
 '757': 'Separating, filtering, and clarifying machine operators',
 '758': 'Compressing and compacting machine operators',
 '759': 'Painting and paint spraying machine operators',
 '763': 'Roasting and baking machine operators, food',
 '764': 'Washing, cleaning, and pickling machine operators',
 '765': 'Folding machine operators',
 '766': 'Furnace, kiln, and oven operators, except food',
 '768': 'Crushing and grinding machine operators',
 '769': 'Slicing and cutting machine operators',
 '773': 'Motion picture projectionists',
 '774': 'Photographic process machine operators',
 '777': 'Miscellaneous machine operators, n.e.c.',
 '779': 'Machine operators, n.s. ',
 '783': 'Welders and cutters',
 '784': 'Solderers and brazers',
 '785': 'Assemblers',
 '786': 'Hand cutting and trimming occupations',
 '787': 'Hand molding, casting, and forming occupations',
 '789': 'Hand painting, coating, and decorating occupations',
 '793': 'Hand engraving and printing occupations',
 '795': 'Miscellaneous hand working occupations ',
 '796': 'Production inspectors, checkers, and examiners',
 '797': 'Production testers',
 '798': 'Production samplers and weighers',
 '799': 'Graders and sorters, except agricultural ',
 '803': 'Supervisors, motor vehicle operators',
 '804': 'Truck drivers',
 '806': 'Driver-sales workers',
 '808': 'Bus drivers',
 '809': 'Taxicab drivers and chauffeurs',
 '813': 'Parking lot attendants',
 '814': 'Motor transportation occupations, n.e.c.',
 '815': 'Transport equipment operatives -- allocat',
 '823': 'Railroad conductors and yardmasters',
 '824': 'Locomotive operating occupations',
 '825': 'Railroad brake, signal, and switch operators',
 '826': 'Rail vehicle operators, n.e.c.',
 '828': 'Ship captains and mates, except fishing boats',
 '829': 'Sailors and deckhands',
 '833': 'Marine engineers',
 '834': 'Bridge, lock, and lighthouse tenders',
 '843': 'Supervisors, material moving equipment operators',
 '844': 'Operating engineers',
 '845': 'Longshore equipment operators',
 '848': 'Hoist and winch operators',
 '849': 'Crane and tower operators',
 '853': 'Excavating and loading machine operators',
 '855': 'Grader, dozer, and scraper operators',
 '856': 'Industrial truck and tractor equipment operators',
 '859': 'Miscellaneous material moving equipment operators',
 '864': 'Supervisors, handlers, equipment cleaners, and laborers, n.e.c.',
 '865': 'Helpers, mechanics and repairers',
 '866': 'Helpers, construction trades',
 '867': 'Helpers, surveyor',
 '868': 'Helpers, extractive occupations',
 '869': 'Construction laborers',
 '874': 'Production helpers',
 '875': 'Garbage collectors',
 '876': 'Stevedores',
 '877': 'Stock handlers and baggers',
 '878': 'Machine feeders and offbearers',
 '883': 'Freight, stock, and material handlers, n.e.c.',
 '885': 'Garage and service station related occupations',
 '887': 'Vehicle washers and equipment cleaners',
 '888': 'Hand packers and packagers',
 '889': 'Laborers, except construction',
 '890': 'Laborers, except farm -- allocat',
 '903': 'Commissioned ',
 '904': 'Non-commission',
 '905': 'Military occupation, rank not specified ',
 '909': 'Last worked'}

In [4]:
occ1950_map = {'000': 'Accountants and auditors',
 '001': 'Actors and actresses',
 '002': 'Airplane pilots and navigators',
 '003': 'Architects',
 '004': 'Artists and art teachers',
 '005': 'Athletes',
 '006': 'Authors',
 '007': 'Chemists',
 '008': 'Chiropractors',
 '009': 'Clergymen',
 '010': 'College presidents and deans',
 '012': 'Agricultural sciences',
 '013': 'Biological sciences',
 '014': 'Chemistry',
 '015': 'Economics',
 '016': 'Engineering',
 '017': 'Geology and geophysics',
 '018': 'Mathematics',
 '019': 'Medical sciences',
 '023': 'Physics',
 '024': 'Psychology',
 '025': 'Statistics',
 '026': 'Natural science (n.e.c.)',
 '027': 'Social sciences (n.e.c.)',
 '028': 'Non-scientific subjects',
 '029': 'Subject not specified',
 '031': 'Dancers and dancing teachers',
 '032': 'Dentists',
 '033': 'Designers',
 '034': 'Dietitians and nutritionists',
 '035': 'Draftsmen',
 '036': 'Editors and reporters',
 '041': 'Engineers, aeronautical',
 '042': 'Engineers, chemical',
 '043': 'Engineers, civil',
 '044': 'Engineers, electrical',
 '045': 'Engineers, industrial',
 '046': 'Engineers, mechanical',
 '047': 'Engineers, metallurgical, metallurgists',
 '048': 'Engineers, mining',
 '049': 'Engineers (n.e.c.)',
 '051': 'Entertainers (n.e.c.)',
 '052': 'Farm and home management advisors',
 '053': 'Foresters and conservationists',
 '054': 'Funeral directors and embalmers',
 '055': 'Lawyers and judges',
 '056': 'Librarians',
 '057': 'Musicians and music teachers',
 '058': 'Nurses, professional',
 '059': 'Nurses, student professional',
 '061': 'Agricultural scientists',
 '062': 'Biological scientists',
 '063': 'Geologists and geophysicists',
 '067': 'Mathematicians',
 '068': 'Physicists',
 '069': 'Miscellaneous natural scientists',
 '070': 'Optometrists',
 '071': 'Osteopaths',
 '072': 'Personnel and labor relations workers',
 '073': 'Pharmacists',
 '074': 'Photographers',
 '075': 'Physicians and surgeons',
 '076': 'Radio operators',
 '077': 'Recreation and group workers',
 '078': 'Religious workers',
 '079': 'Social and welfare workers, except group',
 '081': 'Economists',
 '082': 'Psychologists',
 '083': 'Statisticians and actuaries',
 '084': 'Miscellaneous social scientists',
 '091': 'Sports instructors and officials',
 '092': 'Surveyors',
 '093': 'Teachers (n.e.c.)',
 '094': 'Technicians, medical and dental',
 '095': 'Technicians, testing',
 '096': 'Technicians (n.e.c.)',
 '097': 'Therapists and healers (n.e.c.)',
 '098': 'Veterinarians',
 '099': 'Professional, technical and kindred workers (n.e.c.)',
 '100': 'Farmers (owners and tenants)',
 '123': 'Farm managers',
 '200': 'Buyers and department heads, store',
 '201': 'Buyers and shippers, farm products',
 '203': 'Conductors, railroad',
 '204': 'Credit men',
 '205': 'Floormen and floor managers, store',
 '210': 'Inspectors, public administration',
 '230': 'Managers and superintendents, building',
 '240': 'Officers, pilots, pursers and engineers, ship',
 '250': 'Officials and administrators (n.e.c.), public administration',
 '260': 'Officials, lodge, society, union, etc.',
 '270': 'Postmasters',
 '280': 'Purchasing agents and buyers (n.e.c.)',
 '290': 'Managers, officials, and proprietors (n.e.c.)',
 '300': 'Agents (n.e.c.)',
 '301': 'Attendants and assistants, library',
 '302': "Attendants, physician's and dentist's office",
 '304': 'Baggagemen, transportation',
 '305': 'Bank tellers',
 '310': 'Bookkeepers',
 '320': 'Cashiers',
 '321': 'Collectors, bill and account',
 '322': 'Dispatchers and starters, vehicle',
 '325': 'Express messengers and railway mail clerks',
 '335': 'Mail carriers',
 '340': 'Messengers and office boys',
 '341': 'Office machine operators',
 '342': 'Shipping and receiving clerks',
 '350': 'Stenographers, typists, and secretaries',
 '360': 'Telegraph messengers',
 '365': 'Telegraph operators',
 '370': 'Telephone operators',
 '380': 'Ticket, station, and express agents',
 '390': 'Clerical and kindred workers (n.e.c.)',
 '400': 'Advertising agents and salesmen',
 '410': 'Auctioneers',
 '420': 'Demonstrators',
 '430': 'Hucksters and peddlers',
 '450': 'Insurance agents and brokers',
 '460': 'Newsboys',
 '470': 'Real estate agents and brokers',
 '480': 'Stock and bond salesmen',
 '490': 'Salesmen and sales clerks (n.e.c.)',
 '500': 'Bakers',
 '501': 'Blacksmiths',
 '502': 'Bookbinders',
 '503': 'Boilermakers',
 '504': 'Brickmasons, stonemasons, and tile setters',
 '505': 'Cabinetmakers',
 '510': 'Carpenters',
 '511': 'Cement and concrete finishers',
 '512': 'Compositors and typesetters',
 '513': 'Cranemen, derrickmen, and hoistmen',
 '514': 'Decorators and window dressers',
 '515': 'Electricians',
 '520': 'Electrotypers and stereotypers',
 '521': 'Engravers, except photoengravers',
 '522': 'Excavating, grading, and road machinery operators',
 '523': 'Foremen (n.e.c.)',
 '524': 'Forgemen and hammermen',
 '525': 'Furriers',
 '530': 'Glaziers',
 '531': 'Heat treaters, annealers, temperers',
 '532': 'Inspectors, scalers, and graders, log and lumber',
 '533': 'Inspectors (n.e.c.)',
 '534': 'Jewelers, watchmakers, goldsmiths, and silversmiths',
 '535': 'Job setters, metal',
 '540': 'Linemen and servicemen, telegraph, telephone, and power',
 '541': 'Locomotive engineers',
 '542': 'Locomotive firemen',
 '543': 'Loom fixers',
 '544': 'Machinists',
 '545': 'Mechanics and repairmen, airplane',
 '550': 'Mechanics and repairmen, automobile',
 '551': 'Mechanics and repairmen, office machine',
 '552': 'Mechanics and repairmen, radio and television',
 '553': 'Mechanics and repairmen, railroad and car shop',
 '554': 'Mechanics and repairmen (n.e.c.)',
 '555': 'Millers, grain, flour, feed, etc.',
 '560': 'Millwrights',
 '561': 'Molders, metal',
 '562': 'Motion picture projectionists',
 '563': 'Opticians and lens grinders and polishers',
 '564': 'Painters, construction and maintenance',
 '565': 'Paperhangers',
 '570': 'Pattern and model makers, except paper',
 '571': 'Photoengravers and lithographers',
 '572': 'Piano and organ tuners and repairmen',
 '573': 'Plasterers',
 '574': 'Plumbers and pipe fitters',
 '575': 'Pressmen and plate printers, printing',
 '580': 'Rollers and roll hands, metal',
 '581': 'Roofers and slaters',
 '582': 'Shoemakers and repairers, except factory',
 '583': 'Stationary engineers',
 '584': 'Stone cutters and stone carvers',
 '585': 'Structural metal workers',
 '590': 'Tailors and tailoresses',
 '591': 'Tinsmiths, coppersmiths, and sheet metal workers',
 '592': 'Tool makers, and die makers and setters',
 '593': 'Upholsterers',
 '594': 'Craftsmen and kindred workers (n.e.c.)',
 '595': 'Members of the armed services',
 '600': 'Apprentice auto mechanics',
 '601': 'Apprentice bricklayers and masons',
 '602': 'Apprentice carpenters',
 '603': 'Apprentice electricians',
 '604': 'Apprentice machinists and toolmakers',
 '605': 'Apprentice mechanics, except auto',
 '610': 'Apprentice plumbers and pipe fitters',
 '611': 'Apprentices, building trades (n.e.c.)',
 '612': 'Apprentices, metalworking trades (n.e.c.)',
 '613': 'Apprentices, printing trades',
 '614': 'Apprentices, other specified trades',
 '615': 'Apprentices, trade not specified',
 '620': 'Asbestos and insulation workers',
 '621': 'Attendants, auto service and parking',
 '622': 'Blasters and powdermen',
 '623': 'Boatmen, canalmen, and lock keepers',
 '624': 'Brakemen, railroad',
 '625': 'Bus drivers',
 '630': 'Chainmen, rodmen, and axmen, surveying',
 '631': 'Conductors, bus and street railway',
 '632': 'Deliverymen and routemen',
 '633': 'Dressmakers and seamstresses, except factory',
 '634': 'Dyers',
 '635': 'Filers, grinders, and polishers, metal',
 '640': 'Fruit, nut, and vegetable graders, and packers, except factory',
 '641': 'Furnacemen, smeltermen and pourers',
 '642': 'Heaters, metal',
 '643': 'Laundry and dry cleaning operatives',
 '644': 'Meat cutters, except slaughter and packing house',
 '645': 'Milliners',
 '650': 'Mine operatives and laborers',
 '660': 'Motormen, mine, factory, logging camp, etc.',
 '661': 'Motormen, street, subway, and elevated railway',
 '662': 'Oilers and greaser, except auto',
 '670': 'Painters, except construction or maintenance',
 '671': 'Photographic process workers',
 '672': 'Power station operators',
 '673': 'Sailors and deck hands',
 '674': 'Sawyers',
 '675': 'Spinners, textile',
 '680': 'Stationary firemen',
 '681': 'Switchmen, railroad',
 '682': 'Taxicab drivers and chauffers',
 '683': 'Truck and tractor drivers',
 '684': 'Weavers, textile',
 '685': 'Welders and flame cutters',
 '690': 'Operative and kindred workers (n.e.c.)',
 '700': 'Housekeepers, private household',
 '710': 'Laundresses, private household',
 '720': 'Private household workers (n.e.c.)',
 '730': 'Attendants, hospital and other institution',
 '731': 'Attendants, professional and personal service (n.e.c.)',
 '732': 'Attendants, recreation and amusement',
 '740': 'Barbers, beauticians, and manicurists',
 '750': 'Bartenders',
 '751': 'Bootblacks',
 '752': 'Boarding and lodging house keepers',
 '753': 'Charwomen and cleaners',
 '754': 'Cooks, except private household',
 '760': 'Counter and fountain workers',
 '761': 'Elevator operators',
 '762': 'Firemen, fire protection',
 '763': 'Guards, watchmen, and doorkeepers',
 '764': 'Housekeepers and stewards, except private household',
 '770': 'Janitors and sextons',
 '771': 'Marshals and constables',
 '772': 'Midwives',
 '773': 'Policemen and detectives',
 '780': 'Porters',
 '781': 'Practical nurses',
 '782': 'Sheriffs and bailiffs',
 '783': 'Ushers, recreation and amusement',
 '784': 'Waiters and waitresses',
 '785': 'Watchmen (crossing) and bridge tenders',
 '790': 'Service workers, except private household (n.e.c.)',
 '810': 'Farm foremen',
 '820': 'Farm laborers, wage workers',
 '830': 'Farm laborers, unpaid family workers',
 '840': 'Farm service laborers, self-employed',
 '910': 'Fishermen and oystermen',
 '920': 'Garage laborers and car washers and greasers',
 '930': 'Gardeners, except farm and groundskeepers',
 '940': 'Longshoremen and stevedores',
 '950': 'Lumbermen, raftsmen, and woodchoppers',
 '960': 'Teamsters',
 '970': 'Laborers (n.e.c.)',
 '979': 'Not yet classified',
 '980': 'Keeps house/housekeeping at home/housewife',
 '981': 'Imputed keeping house (1850-1900)',
 '982': 'Helping at home/helps parents/housework',
 '983': 'At school/student',
 '984': 'Retired',
 '985': 'Unemployed/without occupation',
 '986': 'Invalid/disabled w/ no occupation reported',
 '987': 'Inmate',
 '990': 'New Worker',
 '991': 'Gentleman/lady/at leisure',
 '995': 'Other non-occupational response',
 '997': 'Occupation missing/unknown',
 '999': 'N/A (blank)'}